In [94]:
import sys

sys.path.insert(
    1, "..\\utilities\\"
)  # adds utilities folder to path so we can import modules from it, won't be needed after packaging

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import loading_utils as load

participant_list = [200, 201, 202, 204, 205, 206, 207, 209, 210, 211, 212, 213]

In [95]:
data_dir = "./results/new/resampled/"
trial_stats_dict = {}
stim_stats_dict = {}
for participant_id in participant_list:
    data_path = os.path.join(data_dir, str(participant_id) + "_resampled_data.csv")
    data_df_new = pd.read_csv(data_path)

    groupby_df = (
        data_df_new[
            ["Block", "Trial no", "Trial type", "Trial phase", "Stim eye - Size Mm"]
        ]
        .groupby(["Block", "Trial no", "Trial type"])
        .agg(["count", "size"])
        .reset_index()
    )
    groupby_df[("Stim eye - Size Mm", "count/size ratio")] = (
        groupby_df[("Stim eye - Size Mm", "count")]
        / groupby_df[("Stim eye - Size Mm", "size")]
    )
    groupby_df[("Stim eye - Size Mm", "more than 50%")] = (
        groupby_df[("Stim eye - Size Mm", "count/size ratio")] >= 0.5
    )
    trial_stats_dict[str(participant_id)] = (
        groupby_df[
            [
                ("Block", ""),
                ("Trial no", ""),
                ("Trial type", ""),
                ("Stim eye - Size Mm", "count/size ratio"),
                ("Stim eye - Size Mm", "more than 50%"),
            ]
        ]
        .groupby([("Block", ""), ("Trial type", "")])
        .agg(
            {
                ("Stim eye - Size Mm", "count/size ratio"): [
                    "mean",
                    "std",
                    "min",
                    "max",
                ],
                ("Stim eye - Size Mm", "more than 50%"): "sum",
                ("Trial no", ""): "count",
            }
        )
        .reset_index()
    )
    
    #now for only stimulus phase
    
    data_df_new = data_df_new[data_df_new['Trial phase']=='stim']
    groupby_df = (
        data_df_new[
            ["Block", "Trial no", "Trial type", "Stim eye - Size Mm"]
        ]
        .groupby(["Block", "Trial no", "Trial type"])
        .agg(["count", "size"])
        .reset_index()
    )
    groupby_df[("Stim eye - Size Mm", "count/size ratio")] = (
        groupby_df[("Stim eye - Size Mm", "count")]
        / groupby_df[("Stim eye - Size Mm", "size")]
    )
    groupby_df[("Stim eye - Size Mm", "more than 50%")] = (
        groupby_df[("Stim eye - Size Mm", "count/size ratio")] >= 0.5
    )
    stim_stats_dict[str(participant_id)] = (
        groupby_df[
            [
                ("Block", ""),
                ("Trial no", ""),
                ("Trial type", ""),
                ("Stim eye - Size Mm", "count/size ratio"),
                ("Stim eye - Size Mm", "more than 50%"),
            ]
        ]
        .groupby([("Block", ""), ("Trial type", "")])
        .agg(
            {
                ("Stim eye - Size Mm", "count/size ratio"): [
                    "mean",
                    "std",
                    "min",
                    "max",
                ],
                ("Stim eye - Size Mm", "more than 50%"): "sum",
                ("Trial no", ""): "count",
            }
        )
        .reset_index()
    )
    data_df_new = []
    groupby_df = []

## Summary:

I did statistics per block, per condition for three types of datasets:

1. Resampled to 50 Hz
2. NaNs resulting from non-stimulated eye measurement removed, resampled to 50 Hz.
3. NaNs resulting from non-stimulated eye measurement removed, resampled to 25 Hz.

There isn't a visible difference between 1 and 2, which was to be expected, since resampling uses mean without NaNs and doesn't fill them. Resampling to 25 Hz makes the good recordings have up to 100% samples complete, so only minimal interpolation would be needed. Analysis methods will need to be laid out clearly to figure out the optimal sampling frequency, taking the unreliability of the instrument into account.

A check of NaN sequences prevalence and length may be beneficial, even when resampling to 25 Hz: a trial with 90% of data present may still be not usable if a lot of data is missing in a concentrated period of interest.

Here I list blocks per participant that have data completeness issues with short commentary based on statistics for 50 Hz resampled data:

Participant 200: over 50% data remaining in almost all trials with at most 1-2 exceptions per block.

Participant 201: block 0: only 5 trials per condition. (need to double check if nothing went wrong with pulling data), block 5: about 5/10 trials at more than 50% per cond.

Participant 202: block 2: about 5/10 trials at more than 50% per cond., block 10: about 13-15 trials out of 20  at more than 50% per cond.

Participant 204: blocks 0, 5, 6, 7, 8, 9: almost no trials at more than 50% available data per cond.

Participant 205: sparse data, almost all of it is at much below 50% data available.

Participant 206: block 1: almost no data at >50%. Most blocks have more than 5 trials per condition at more than 50% data available.

Participant 207: blocks 2,3,4,5,6,10: close to 0 trials with more than 50% data.

Participant 209: only block 5 has fairly complete trials (more than 50% data).

Participant 210: most blocks have few or zero trials with more than 50% data available.

Participant 211: blocks 5, 6, 7: about half the trials with more than 50% data per cond.

Participant 212: blocks 0, 1, 6: about half the trials with more than 50% data per cond.

Participant 213: blocks 0,5,6,7,8: close to 0 trials with more than 50% data remaining, blocks 9,10: about half of trials.

Global conclusion: if data is missing, it's usually missing in the whole block to a similar level. Participants 204, 205, 207, 209, 210, 213 have the most missing data. 

Analysis in current state should be possible without larger issues for 200, 201, 202, 211, 212, especially if data were to be downsampled more (e.g. to 25 - 30 Hz). For 206, block 1 may need to be excluded.


## Statistics of available data for each participant per block, per stimulus

count/size ratio - number of samples without nan measurements divided by number of all samples

The above is calculated for each trial, grouped by trial type and trial phase. Then the mean, standard deviation, minimum and maximum are calculated for each group.

more than 50% - count - number of trials with count/size ratio higher or equal to 0.5

Trial no - count - overall number of trials

### Participant 200

In [96]:
print('Statistics for whole trials')
trial_stats_dict["200"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.585586  0.002329  0.580184  0.588175   
1      0        l-m           0.586488  0.001821  0.582227  0.588355   
2      0        lms           0.587072  0.001605  0.583673  0.588957   
3      0        mel           0.585689  0.002094  0.582227  0.588776   
4      0          s           0.586675  0.001264  0.583673  0.588175   
5      1       flux           0.585222  0.003024  0.577551  0.588355   
6      1        l-m           0.586165  0.001322  0.583078  0.587334   
7      1        lms           0.586926  0.001700  0.582653  0.589377   
8      1        mel           0.586932  0.001138  0.584867  0.588355   
9      1          s           0.585569  0.001390  0.582822  0.587156   
10     2       flux           0.584346  0.002401  0.580020  0.587156   
11     2        l-m           0.585527  0.003524  0.576531  0.588776   
12     2        lms           0.585912  0.001373  0.582653  0.587334   
13     2        mel           0.584176  0.004378  0.574055  0.588355   
14     2          s           0.584286  0.005321  0.572449  0.587755   
15     3       flux           0.587287  0.001233  0.584694  0.588776   
16     3        l-m           0.586837  0.001721  0.583673  0.588776   
17     3        lms           0.587577  0.001110  0.585890  0.589377   
18     3        mel           0.587281  0.001088  0.585291  0.589377   
19     3          s           0.586200  0.001617  0.582653  0.587755   
20     4       flux           0.586591  0.001416  0.583673  0.588355   
21     4        l-m           0.585774  0.002568  0.581205  0.588776   
22     4        lms           0.586969  0.001077  0.584694  0.588355   
23     4        mel           0.586668  0.001503  0.584270  0.588355   
24     4          s           0.585588  0.003371  0.576531  0.587755   
25     5       flux           0.583920  0.003722  0.577982  0.588175   
26     5        l-m           0.583036  0.007311  0.563265  0.588355   
27     5        lms           0.584327  0.003009  0.577982  0.587334   
28     5        mel           0.584548  0.003450  0.579162  0.588175   
29     5          s           0.582381  0.004296  0.573904  0.586735   
30     6       flux           0.585876  0.002560  0.582227  0.588776   
31     6        l-m           0.581854  0.005783  0.569388  0.588355   
32     6        lms           0.583760  0.005445  0.569827  0.587755   
33     6        mel           0.585774  0.002859  0.578571  0.588776   
34     6          s           0.573121  0.037136  0.468367  0.588776   
35     7       flux           0.587245  0.001004  0.585714  0.588776   
36     7        l-m           0.586896  0.000841  0.585291  0.588175   
37     7        lms           0.587475  0.000881  0.586106  0.588957   
38     7        mel           0.586992  0.001217  0.584270  0.588355   
39     7          s           0.586429  0.001408  0.583673  0.588175   
40     8       flux           0.584995  0.003726  0.573034  0.587334   
41     8        l-m           0.586815  0.001771  0.583078  0.589377   
42     8        lms           0.549629  0.141078  0.039712  0.588776   
43     8        mel           0.585686  0.002122  0.579592  0.588776   
44     8          s           0.585222  0.002550  0.581040  0.588776   
45     9       flux           0.580786  0.008789  0.564730  0.587755   
46     9        l-m           0.580732  0.007102  0.568648  0.588175   
47     9        lms           0.579903  0.011237  0.556345  0.589195   
48     9        mel           0.583471  0.005017  0.572895  0.586735   
49     9          s           0.581723  0.008170  0.567623  0.588776   
50    10       flux           0.586343  0.002377  0.580777  0.588776   
51    10        l-m           0.587756  0.000945  0.586137  0.588776   
52    10        lms           0.585348  0.0

In [97]:
print('Statistics for stimulus phase')
stim_stats_dict["200"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.588048  0.002057  0.585657  0.589641   
1      0        l-m           0.586853  0.002689  0.581673  0.589641   
2      0        lms           0.586454  0.003143  0.581673  0.589641   
3      0        mel           0.587649  0.003872  0.577689  0.589641   
4      0          s           0.586056  0.003488  0.581673  0.589641   
5      1       flux           0.583267  0.012055  0.549801  0.589641   
6      1        l-m           0.586853  0.002689  0.581673  0.589641   
7      1        lms           0.588446  0.002689  0.585657  0.593625   
8      1        mel           0.587251  0.002786  0.581673  0.589641   
9      1          s           0.584064  0.005039  0.573705  0.589641   
10     2       flux           0.584462  0.010123  0.557769  0.593625   
11     2        l-m           0.587251  0.002057  0.585657  0.589641   
12     2        lms           0.583665  0.004695  0.577689  0.589641   
13     2        mel           0.584861  0.007225  0.565737  0.589641   
14     2          s           0.585259  0.004770  0.573705  0.589641   
15     3       flux           0.588845  0.001680  0.585657  0.589641   
16     3        l-m           0.586056  0.004384  0.577689  0.589641   
17     3        lms           0.588048  0.002786  0.581673  0.589641   
18     3        mel           0.588446  0.001924  0.585657  0.589641   
19     3          s           0.587251  0.003360  0.581673  0.589641   
20     4       flux           0.586056  0.004770  0.577689  0.589641   
21     4        l-m           0.586056  0.003488  0.577689  0.589641   
22     4        lms           0.588446  0.002689  0.585657  0.593625   
23     4        mel           0.586056  0.003962  0.577689  0.589641   
24     4          s           0.582869  0.007979  0.561753  0.589641   
25     5       flux           0.585259  0.004770  0.577689  0.589641   
26     5        l-m           0.583267  0.008441  0.561753  0.589641   
27     5        lms           0.584462  0.008617  0.561753  0.589641   
28     5        mel           0.588845  0.001680  0.585657  0.589641   
29     5          s           0.583267  0.005697  0.573705  0.589641   
30     6       flux           0.588845  0.001680  0.585657  0.589641   
31     6        l-m           0.582470  0.011699  0.561753  0.593625   
32     6        lms           0.586853  0.004221  0.577689  0.589641   
33     6        mel           0.585657  0.005634  0.573705  0.589641   
34     6          s           0.552590  0.111602  0.235060  0.589641   
35     7       flux           0.587251  0.002786  0.581673  0.589641   
36     7        l-m           0.584861  0.002520  0.581673  0.589641   
37     7        lms           0.587649  0.003872  0.581673  0.593625   
38     7        mel           0.587251  0.003360  0.581673  0.589641   
39     7          s           0.585259  0.004770  0.573705  0.589641   
40     8       flux           0.585923  0.003829  0.577689  0.589641   
41     8        l-m           0.585923  0.004115  0.577689  0.589641   
42     8        lms           0.567331  0.074334  0.298805  0.589641   
43     8        mel           0.586189  0.003647  0.577689  0.589641   
44     8          s           0.586189  0.004224  0.577689  0.589641   
45     9       flux           0.579681  0.010157  0.561753  0.589641   
46     9        l-m           0.580478  0.009947  0.565737  0.589641   
47     9        lms           0.570916  0.027923  0.498008  0.589641   
48     9        mel           0.584064  0.009795  0.557769  0.589641   
49     9          s           0.578884  0.012323  0.557769  0.589641   
50    10       flux           0.584861  0.005245  0.573705  0.589641   
51    10        l-m           0.588048  0.002057  0.585657  0.589641   
52    10        lms           0.584064  0.0

### Participant 201

In [98]:
print('Statistics for whole trials')
trial_stats_dict["201"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.528952  0.057842  0.445918  0.586510   
1      0        l-m           0.525290  0.092868  0.360572  0.586735   
2      0        lms           0.439848  0.155516  0.235714  0.578571   
3      0        mel           0.424165  0.106904  0.297959  0.559184   
4      0          s           0.475015  0.100987  0.356122  0.585714   
5      1       flux           0.571568  0.014195  0.541544  0.586735   
6      1        l-m           0.573634  0.008165  0.559063  0.584270   
7      1        lms           0.569588  0.015842  0.542390  0.584149   
8      1        mel           0.568938  0.014017  0.532990  0.582653   
9      1          s           0.558986  0.026414  0.497477  0.585714   
10     2       flux           0.577925  0.011421  0.547959  0.587755   
11     2        l-m           0.580958  0.007548  0.563265  0.587755   
12     2        lms           0.576822  0.013501  0.551859  0.588355   
13     2        mel           0.575147  0.010388  0.549540  0.586313   
14     2          s           0.580280  0.006163  0.566769  0.587755   
15     3       flux           0.572058  0.009924  0.556122  0.586735   
16     3        l-m           0.569100  0.019020  0.544343  0.588776   
17     3        lms           0.570045  0.012772  0.552605  0.586313   
18     3        mel           0.575866  0.014056  0.552147  0.587755   
19     3          s           0.576201  0.015484  0.534694  0.587334   
20     4       flux           0.532701  0.068313  0.373469  0.587755   
21     4        l-m           0.563742  0.029895  0.505607  0.587935   
22     4        lms           0.521270  0.103613  0.272727  0.589377   
23     4        mel           0.550084  0.057988  0.393878  0.588595   
24     4          s           0.553318  0.042699  0.459735  0.587156   
25     5       flux           0.466989  0.119508  0.228805  0.580235   
26     5        l-m           0.398179  0.179454  0.147959  0.567347   
27     5        lms           0.432097  0.125422  0.235714  0.580612   
28     5        mel           0.456648  0.110467  0.267620  0.587755   
29     5          s           0.454585  0.147823  0.142857  0.582059   
30     6       flux           0.577821  0.006419  0.564862  0.583673   
31     6        l-m           0.568829  0.016920  0.537755  0.586735   
32     6        lms           0.573445  0.015987  0.539246  0.586735   
33     6        mel           0.576782  0.010998  0.553626  0.585714   
34     6          s           0.564820  0.014452  0.539326  0.581040   
35     7       flux           0.569319  0.022867  0.528571  0.588175   
36     7        l-m           0.563531  0.041798  0.452041  0.587755   
37     7        lms           0.546749  0.061402  0.411645  0.587334   
38     7        mel           0.549516  0.059232  0.419816  0.586735   
39     7          s           0.530555  0.091797  0.327551  0.587755   
40     8       flux           0.585793  0.001560  0.583078  0.588175   
41     8        l-m           0.582527  0.010681  0.553061  0.588776   
42     8        lms           0.583531  0.004317  0.575510  0.587156   
43     8        mel           0.584159  0.006866  0.565306  0.589796   
44     8          s           0.580266  0.011444  0.559184  0.589980   
45     9       flux           0.580716  0.011221  0.558163  0.588355   
46     9        l-m           0.586266  0.002833  0.579162  0.589377   
47     9        lms           0.584736  0.001931  0.581633  0.588355   
48     9        mel           0.583179  0.007248  0.568367  0.588355   
49     9          s           0.584949  0.005083  0.570991  0.588355   
50    10       flux           0.578082  0.009877  0.554536  0.587935   
51    10        l-m           0.579001  0.007711  0.563710  0.587935   
52    10        lms           0.573056  0.0

In [99]:
print('Statistics for stimulus phase')
stim_stats_dict["201"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.560159  0.061502  0.450199  0.589641   
1      0        l-m           0.564143  0.054815  0.466135  0.589641   
2      0        lms           0.546614  0.060513  0.442231  0.589641   
3      0        mel           0.415139  0.240226  0.003984  0.589641   
4      0          s           0.562550  0.060644  0.454183  0.593625   
5      1       flux           0.577291  0.026789  0.501992  0.589641   
6      1        l-m           0.579681  0.010499  0.561753  0.593625   
7      1        lms           0.581275  0.009659  0.561753  0.589641   
8      1        mel           0.580876  0.007225  0.565737  0.589641   
9      1          s           0.580478  0.008197  0.561753  0.589641   
10     2       flux           0.584462  0.004620  0.573705  0.589641   
11     2        l-m           0.586454  0.003143  0.581673  0.589641   
12     2        lms           0.588048  0.002786  0.581673  0.589641   
13     2        mel           0.586056  0.004770  0.577689  0.593625   
14     2          s           0.586454  0.006452  0.569721  0.593625   
15     3       flux           0.586056  0.003962  0.577689  0.589641   
16     3        l-m           0.584861  0.011547  0.553785  0.589641   
17     3        lms           0.585259  0.007382  0.565737  0.593625   
18     3        mel           0.588446  0.001924  0.585657  0.589641   
19     3          s           0.586853  0.004987  0.573705  0.589641   
20     4       flux           0.584462  0.009017  0.561753  0.593625   
21     4        l-m           0.584064  0.013569  0.545817  0.589641   
22     4        lms           0.568924  0.045340  0.442231  0.589641   
23     4        mel           0.587251  0.002786  0.581673  0.589641   
24     4          s           0.568526  0.036662  0.478088  0.589641   
25     5       flux           0.533466  0.063452  0.442231  0.585657   
26     5        l-m           0.504382  0.117246  0.306773  0.593625   
27     5        lms           0.531474  0.078639  0.370518  0.589641   
28     5        mel           0.491235  0.113638  0.286853  0.589641   
29     5          s           0.543028  0.076544  0.410359  0.593625   
30     6       flux           0.587649  0.002817  0.581673  0.589641   
31     6        l-m           0.586853  0.003280  0.581673  0.589641   
32     6        lms           0.586454  0.003661  0.581673  0.589641   
33     6        mel           0.588845  0.002520  0.585657  0.593625   
34     6          s           0.588446  0.002689  0.585657  0.593625   
35     7       flux           0.566534  0.044317  0.450199  0.589641   
36     7        l-m           0.586853  0.002689  0.581673  0.589641   
37     7        lms           0.523904  0.124682  0.266932  0.589641   
38     7        mel           0.521514  0.139049  0.215139  0.589641   
39     7          s           0.494821  0.182416  0.147410  0.589641   
40     8       flux           0.586853  0.004221  0.581673  0.593625   
41     8        l-m           0.586056  0.005126  0.573705  0.589641   
42     8        lms           0.586454  0.002520  0.581673  0.589641   
43     8        mel           0.587649  0.004303  0.577689  0.593625   
44     8          s           0.586853  0.003780  0.577689  0.589641   
45     9       flux           0.586454  0.002520  0.581673  0.589641   
46     9        l-m           0.588048  0.002786  0.581673  0.589641   
47     9        lms           0.586056  0.003488  0.581673  0.589641   
48     9        mel           0.586454  0.004115  0.581673  0.593625   
49     9          s           0.587649  0.003386  0.581673  0.589641   
50    10       flux           0.585657  0.005634  0.577689  0.589641   
51    10        l-m           0.585259  0.008067  0.565737  0.589641   
52    10        lms           0.582072  0.0

### Participant 202

In [100]:
print('Statistics for whole trials')
trial_stats_dict["202"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.574079  0.021569  0.526531  0.588776   
1      0        l-m           0.584167  0.003841  0.576531  0.587755   
2      0        lms           0.581327  0.013478  0.546476  0.588355   
3      0        mel           0.577857  0.017654  0.532653  0.588355   
4      0          s           0.581938  0.005718  0.570991  0.588776   
5      1       flux           0.569740  0.030290  0.500000  0.588776   
6      1        l-m           0.561076  0.042138  0.462245  0.588175   
7      1        lms           0.572082  0.040542  0.459267  0.589377   
8      1        mel           0.564405  0.072475  0.358163  0.589377   
9      1          s           0.567305  0.049895  0.426531  0.588776   
10     2       flux           0.552419  0.060832  0.388379  0.588776   
11     2        l-m           0.539263  0.084725  0.360572  0.588355   
12     2        lms           0.515374  0.098042  0.291539  0.588063   
13     2        mel           0.565834  0.037941  0.462717  0.588776   
14     2          s           0.544097  0.089436  0.315306  0.587755   
15     3       flux           0.577539  0.010310  0.557594  0.588063   
16     3        l-m           0.584448  0.007050  0.565306  0.589377   
17     3        lms           0.578499  0.010183  0.560204  0.586735   
18     3        mel           0.582424  0.006492  0.570408  0.588175   
19     3          s           0.583954  0.003551  0.576531  0.587334   
20     4       flux           0.575885  0.012976  0.552041  0.587755   
21     4        l-m           0.584023  0.002681  0.579592  0.587334   
22     4        lms           0.573459  0.021675  0.519388  0.587334   
23     4        mel           0.575025  0.021642  0.520940  0.587576   
24     4          s           0.572331  0.020987  0.530071  0.587755   
25     5       flux           0.544629  0.060543  0.440816  0.588355   
26     5        l-m           0.543258  0.066352  0.396534  0.588175   
27     5        lms           0.557543  0.036629  0.498980  0.587755   
28     5        mel           0.561801  0.031813  0.493878  0.587334   
29     5          s           0.550944  0.041118  0.464760  0.587755   
30     6       flux           0.557924  0.053433  0.421859  0.586735   
31     6        l-m           0.541175  0.085256  0.338776  0.588355   
32     6        lms           0.567341  0.034731  0.484200  0.587755   
33     6        mel           0.581326  0.007894  0.567347  0.587755   
34     6          s           0.561019  0.032536  0.505618  0.586735   
35     7       flux           0.584288  0.008875  0.559633  0.588776   
36     7        l-m           0.568325  0.034748  0.498471  0.588776   
37     7        lms           0.585982  0.002872  0.580235  0.588776   
38     7        mel           0.586710  0.001911  0.582227  0.588776   
39     7          s           0.586259  0.002817  0.580184  0.589377   
40     8       flux           0.578191  0.011359  0.553061  0.588776   
41     8        l-m           0.583038  0.010663  0.554082  0.588957   
42     8        lms           0.575072  0.017274  0.536735  0.587156   
43     8        mel           0.579259  0.016443  0.538776  0.588776   
44     8          s           0.579131  0.013385  0.545918  0.587935   
45     9       flux           0.586852  0.001037  0.584694  0.588355   
46     9        l-m           0.586914  0.001622  0.583248  0.588355   
47     9        lms           0.587737  0.001056  0.585533  0.588776   
48     9        mel           0.587298  0.001418  0.585291  0.589377   
49     9          s           0.584694  0.008285  0.561224  0.588355   
50    10       flux           0.451427  0.187667  0.043922  0.587755   
51    10        l-m           0.409758  0.198469  0.041794  0.587755   
52    10        lms           0.450808  0.1

In [101]:
print('Statistics for stimulus phase')
stim_stats_dict["202"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.584064  0.005998  0.569721  0.589641   
1      0        l-m           0.588845  0.003143  0.581673  0.593625   
2      0        lms           0.587251  0.003849  0.577689  0.589641   
3      0        mel           0.572908  0.044674  0.446215  0.589641   
4      0          s           0.583665  0.008023  0.565737  0.589641   
5      1       flux           0.564542  0.048688  0.442231  0.593625   
6      1        l-m           0.545418  0.076137  0.358566  0.589641   
7      1        lms           0.569323  0.054609  0.414343  0.589641   
8      1        mel           0.547410  0.125190  0.191235  0.589641   
9      1          s           0.539044  0.130512  0.171315  0.589641   
10     2       flux           0.529482  0.121010  0.195219  0.589641   
11     2        l-m           0.558167  0.068864  0.366534  0.589641   
12     2        lms           0.482072  0.175419  0.091633  0.593625   
13     2        mel           0.562151  0.054802  0.418327  0.589641   
14     2          s           0.545020  0.097622  0.290837  0.589641   
15     3       flux           0.572908  0.020981  0.537849  0.589641   
16     3        l-m           0.582869  0.018786  0.529880  0.593625   
17     3        lms           0.559761  0.041072  0.482072  0.589641   
18     3        mel           0.577689  0.019061  0.533865  0.589641   
19     3          s           0.582470  0.008769  0.561753  0.589641   
20     4       flux           0.574900  0.019065  0.533865  0.589641   
21     4        l-m           0.585259  0.006627  0.569721  0.589641   
22     4        lms           0.584462  0.008617  0.561753  0.589641   
23     4        mel           0.581673  0.018402  0.529880  0.589641   
24     4          s           0.568526  0.036469  0.482072  0.593625   
25     5       flux           0.535458  0.084623  0.326693  0.593625   
26     5        l-m           0.561753  0.057336  0.406375  0.589641   
27     5        lms           0.565737  0.030458  0.509960  0.589641   
28     5        mel           0.556175  0.045897  0.454183  0.589641   
29     5          s           0.558566  0.050072  0.458167  0.593625   
30     6       flux           0.555777  0.077578  0.338645  0.589641   
31     6        l-m           0.575697  0.025215  0.505976  0.589641   
32     6        lms           0.582869  0.007979  0.561753  0.589641   
33     6        mel           0.587649  0.002817  0.581673  0.589641   
34     6          s           0.582470  0.006719  0.569721  0.589641   
35     7       flux           0.586454  0.006452  0.569721  0.589641   
36     7        l-m           0.549004  0.066608  0.394422  0.589641   
37     7        lms           0.586853  0.004987  0.577689  0.593625   
38     7        mel           0.587649  0.006573  0.569721  0.593625   
39     7          s           0.587251  0.003360  0.581673  0.589641   
40     8       flux           0.574104  0.019468  0.537849  0.589641   
41     8        l-m           0.577689  0.024415  0.517928  0.593625   
42     8        lms           0.577689  0.017916  0.537849  0.593625   
43     8        mel           0.583267  0.009795  0.561753  0.589641   
44     8          s           0.581673  0.014788  0.553785  0.589641   
45     9       flux           0.586853  0.002689  0.581673  0.589641   
46     9        l-m           0.586454  0.003143  0.581673  0.589641   
47     9        lms           0.587649  0.004303  0.577689  0.589641   
48     9        mel           0.586853  0.003780  0.581673  0.589641   
49     9          s           0.584462  0.009586  0.557769  0.589641   
50    10       flux           0.474701  0.187727  0.000000  0.589641   
51    10        l-m           0.438247  0.199748  0.000000  0.589641   
52    10        lms           0.482869  0.1

### Participant 204

In [102]:
print('Statistics for whole trials')
trial_stats_dict["204"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.334070  0.123977  0.159346  0.514872   
1      0        l-m           0.345828  0.120167  0.119388  0.516327   
2      0        lms           0.390501  0.086170  0.253320  0.536188   
3      0        mel           0.349284  0.098104  0.215526  0.503575   
4      0          s           0.344764  0.153032  0.164286  0.559184   
5      1       flux           0.501211  0.066533  0.365949  0.577120   
6      1        l-m           0.476377  0.094352  0.254341  0.567485   
7      1        lms           0.448238  0.137507  0.167347  0.578571   
8      1        mel           0.439074  0.148675  0.208970  0.584270   
9      1          s           0.476320  0.113060  0.250765  0.580020   
10     2       flux           0.485670  0.119136  0.210419  0.578141   
11     2        l-m           0.539411  0.030699  0.495403  0.584694   
12     2        lms           0.499715  0.122291  0.265036  0.583248   
13     2        mel           0.452706  0.110385  0.291837  0.577551   
14     2          s           0.518128  0.067268  0.347959  0.578141   
15     3       flux           0.474850  0.100071  0.288776  0.567347   
16     3        l-m           0.487144  0.043537  0.398573  0.553061   
17     3        lms           0.471448  0.058396  0.348980  0.539796   
18     3        mel           0.506324  0.044296  0.399387  0.571429   
19     3          s           0.489948  0.046085  0.397344  0.548980   
20     4       flux           0.444949  0.086736  0.322778  0.531154   
21     4        l-m           0.408419  0.114031  0.241062  0.548980   
22     4        lms           0.434573  0.147269  0.072449  0.543412   
23     4        mel           0.430944  0.128431  0.146939  0.562245   
24     4          s           0.471809  0.052043  0.375894  0.553517   
25     5       flux           0.404586  0.080117  0.295918  0.560204   
26     5        l-m           0.411503  0.110601  0.167347  0.525510   
27     5        lms           0.354512  0.124758  0.130746  0.505618   
28     5        mel           0.425038  0.041302  0.374490  0.486239   
29     5          s           0.425215  0.107838  0.156282  0.516327   
30     6       flux           0.384883  0.133843  0.185728  0.546939   
31     6        l-m           0.414482  0.101252  0.277835  0.573469   
32     6        lms           0.334238  0.137656  0.070336  0.467824   
33     6        mel           0.436678  0.125645  0.167347  0.566905   
34     6          s           0.363189  0.125731  0.096840  0.511224   
35     7       flux           0.302935  0.109807  0.139869  0.537634   
36     7        l-m           0.277413  0.128025  0.038462  0.487232   
37     7        lms           0.311355  0.161823  0.019608  0.555102   
38     7        mel           0.306101  0.147888  0.064987  0.530612   
39     7          s           0.298390  0.142944  0.093872  0.517347   
40     8       flux           0.359497  0.129894  0.177370  0.556428   
41     8        l-m           0.413041  0.084727  0.276531  0.533129   
42     8        lms           0.414034  0.096308  0.242363  0.560327   
43     8        mel           0.400225  0.071945  0.275510  0.496425   
44     8          s           0.458129  0.092268  0.283963  0.564155   
45     9       flux           0.398910  0.142163  0.068437  0.556751   
46     9        l-m           0.381250  0.127287  0.153218  0.526047   
47     9        lms           0.369758  0.149161  0.039755  0.505102   
48     9        mel           0.400517  0.091577  0.213048  0.510725   
49     9          s           0.363000  0.159594  0.036810  0.546939   
50    10       flux           0.537702  0.023638  0.481595  0.567788   
51    10        l-m           0.530267  0.014651  0.506122  0.554082   
52    10        lms           0.532857  0.0

In [103]:
print('Statistics for stimulus phase')
stim_stats_dict["204"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.305976  0.205863  0.003984  0.577689   
1      0        l-m           0.344622  0.179870  0.031873  0.549801   
2      0        lms           0.448606  0.109418  0.254980  0.585657   
3      0        mel           0.392032  0.109676  0.262948  0.577689   
4      0          s           0.354183  0.192233  0.123506  0.565737   
5      1       flux           0.525896  0.055745  0.410359  0.589641   
6      1        l-m           0.501195  0.082675  0.366534  0.589641   
7      1        lms           0.475299  0.088591  0.270916  0.569721   
8      1        mel           0.509562  0.086198  0.354582  0.589641   
9      1          s           0.490837  0.154060  0.191235  0.589641   
10     2       flux           0.488845  0.174957  0.000000  0.573705   
11     2        l-m           0.543028  0.058374  0.426295  0.593625   
12     2        lms           0.492430  0.099401  0.310757  0.585657   
13     2        mel           0.378486  0.215884  0.000000  0.589641   
14     2          s           0.506773  0.078517  0.342629  0.585657   
15     3       flux           0.498008  0.070322  0.382470  0.589641   
16     3        l-m           0.501195  0.058487  0.398406  0.569721   
17     3        lms           0.545817  0.029035  0.498008  0.577689   
18     3        mel           0.520717  0.062517  0.374502  0.569721   
19     3          s           0.541036  0.054069  0.422311  0.589641   
20     4       flux           0.478088  0.068647  0.334661  0.577689   
21     4        l-m           0.469323  0.121189  0.294821  0.589641   
22     4        lms           0.445817  0.183281  0.000000  0.573705   
23     4        mel           0.480080  0.086622  0.274900  0.569721   
24     4          s           0.523506  0.047035  0.442231  0.593625   
25     5       flux           0.406375  0.149436  0.079681  0.577689   
26     5        l-m           0.466534  0.082539  0.266932  0.537849   
27     5        lms           0.360159  0.193620  0.000000  0.557769   
28     5        mel           0.376892  0.135030  0.167331  0.545817   
29     5          s           0.477689  0.118640  0.195219  0.573705   
30     6       flux           0.411952  0.123130  0.239044  0.573705   
31     6        l-m           0.506375  0.068427  0.366534  0.585657   
32     6        lms           0.374104  0.163206  0.063745  0.509960   
33     6        mel           0.412351  0.146459  0.147410  0.565737   
34     6          s           0.431474  0.166803  0.075697  0.561753   
35     7       flux           0.251262  0.213789  0.000000  0.569721   
36     7        l-m           0.362019  0.298641  0.000000  1.000000   
37     7        lms           0.391235  0.285129  0.000000  1.000000   
38     7        mel           0.320850  0.238777  0.000000  0.549801   
39     7          s           0.317131  0.238283  0.000000  0.569721   
40     8       flux           0.463347  0.067691  0.346614  0.537849   
41     8        l-m           0.499602  0.074255  0.338645  0.573705   
42     8        lms           0.457769  0.113985  0.223108  0.581673   
43     8        mel           0.392430  0.125963  0.147410  0.517928   
44     8          s           0.528287  0.071669  0.390438  0.589641   
45     9       flux           0.424701  0.165893  0.000000  0.561753   
46     9        l-m           0.408367  0.167048  0.123506  0.589641   
47     9        lms           0.407171  0.162885  0.000000  0.533865   
48     9        mel           0.452988  0.056843  0.386454  0.557769   
49     9          s           0.403187  0.177477  0.000000  0.593625   
50    10       flux           0.553386  0.023790  0.505976  0.589641   
51    10        l-m           0.561753  0.015825  0.541833  0.589641   
52    10        lms           0.531474  0.0

### Participant 205

In [104]:
print('Statistics for whole trials')
trial_stats_dict["205"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.291062  0.093891  0.147959  0.395301   
1      0        l-m           0.238022  0.147908  0.046987  0.473469   
2      0        lms           0.231269  0.137532  0.013265  0.436290   
3      0        mel           0.137548  0.114029  0.000000  0.315306   
4      0          s           0.180009  0.118950  0.051020  0.443878   
5      1       flux           0.327086  0.148841  0.164969  0.528571   
6      1        l-m           0.305395  0.132800  0.095821  0.501020   
7      1        lms           0.295430  0.212377  0.000000  0.537755   
8      1        mel           0.270514  0.144867  0.000000  0.439796   
9      1          s           0.312951  0.143791  0.098879  0.548980   
10     2       flux           0.283780  0.207547  0.072375  0.552605   
11     2        l-m           0.270164  0.158526  0.000000  0.482125   
12     2        lms           0.249472  0.184588  0.025536  0.542390   
13     2        mel           0.209806  0.142400  0.058104  0.508163   
14     2          s           0.244097  0.219177  0.010215  0.552605   
15     3       flux           0.285422  0.104082  0.157303  0.502935   
16     3        l-m           0.252136  0.133750  0.101020  0.530612   
17     3        lms           0.334565  0.106393  0.211224  0.518367   
18     3        mel           0.277041  0.159755  0.065306  0.539796   
19     3          s           0.321759  0.123604  0.112245  0.525026   
20     4       flux           0.135704  0.073342  0.036772  0.267347   
21     4        l-m           0.167665  0.158229  0.000000  0.428571   
22     4        lms           0.111500  0.103388  0.000000  0.347959   
23     4        mel           0.160020  0.087891  0.036772  0.335714   
24     4          s           0.136252  0.097183  0.000000  0.272727   
25     5       flux           0.193231  0.150390  0.000000  0.401631   
26     5        l-m           0.229457  0.145345  0.000000  0.435714   
27     5        lms           0.247800  0.133369  0.057201  0.403061   
28     5        mel           0.267909  0.143541  0.082737  0.503061   
29     5          s           0.282882  0.122092  0.109295  0.491318   
30     6       flux           0.346761  0.142909  0.114402  0.512219   
31     6        l-m           0.365870  0.149017  0.141982  0.540816   
32     6        lms           0.370142  0.134795  0.134694  0.552041   
33     6        mel           0.359124  0.131795  0.095918  0.502554   
34     6          s           0.353403  0.123809  0.139653  0.555102   
35     7       flux           0.434127  0.074010  0.348315  0.511747   
36     7        l-m           0.457844  0.034364  0.425510  0.509704   
37     7        lms           0.450803  0.089052  0.352400  0.562072   
38     7        mel           0.331091  0.120379  0.175510  0.469867   
39     7          s           0.387884  0.064706  0.298264  0.474490   
40     8       flux           0.469047  0.053210  0.366972  0.545455   
41     8        l-m           0.405038  0.078608  0.273469  0.511224   
42     8        lms           0.441343  0.093154  0.261224  0.548924   
43     8        mel           0.352323  0.112970  0.201020  0.542857   
44     8          s           0.391748  0.108190  0.229826  0.546381   
45     9       flux           0.299121  0.146356  0.000000  0.482125   
46     9        l-m           0.309950  0.108496  0.150000  0.463812   
47     9        lms           0.355048  0.138963  0.082569  0.508665   
48     9        mel           0.386967  0.088213  0.260204  0.553517   
49     9          s           0.294520  0.122009  0.108053  0.436735   
50    10       flux           0.317908  0.186331  0.043922  0.547959   
51    10        l-m           0.259871  0.161192  0.041794  0.548980   
52    10        lms           0.329607  0.2

In [105]:
print('Statistics for stimulus phase')
stim_stats_dict["205"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.230279  0.237087  0.000000  0.553785   
1      0        l-m           0.270518  0.231270  0.000000  0.569721   
2      0        lms           0.286454  0.185281  0.051793  0.553785   
3      0        mel           0.137052  0.167248  0.000000  0.462151   
4      0          s           0.258167  0.165995  0.000000  0.494024   
5      1       flux           0.308765  0.204310  0.023904  0.577689   
6      1        l-m           0.385259  0.132004  0.123506  0.565737   
7      1        lms           0.340239  0.249493  0.000000  0.585657   
8      1        mel           0.256972  0.234058  0.000000  0.577689   
9      1          s           0.345817  0.156580  0.083665  0.585657   
10     2       flux           0.337052  0.239819  0.000000  0.593625   
11     2        l-m           0.260159  0.161932  0.000000  0.458167   
12     2        lms           0.292032  0.180299  0.087649  0.557769   
13     2        mel           0.292829  0.137939  0.127490  0.513944   
14     2          s           0.269323  0.236904  0.000000  0.585657   
15     3       flux           0.347012  0.207056  0.000000  0.557769   
16     3        l-m           0.284462  0.204862  0.000000  0.557769   
17     3        lms           0.461355  0.095115  0.298805  0.553785   
18     3        mel           0.319522  0.181825  0.000000  0.549801   
19     3          s           0.361753  0.169121  0.000000  0.581673   
20     4       flux           0.144622  0.181828  0.000000  0.505976   
21     4        l-m           0.165339  0.173826  0.000000  0.462151   
22     4        lms           0.167729  0.139833  0.000000  0.434263   
23     4        mel           0.191633  0.126636  0.000000  0.334661   
24     4          s           0.187649  0.163411  0.000000  0.525896   
25     5       flux           0.230279  0.198794  0.000000  0.541833   
26     5        l-m           0.285657  0.145890  0.000000  0.458167   
27     5        lms           0.257371  0.212784  0.000000  0.509960   
28     5        mel           0.313546  0.168403  0.087649  0.509960   
29     5          s           0.264143  0.193819  0.000000  0.545817   
30     6       flux           0.417928  0.132728  0.167331  0.557769   
31     6        l-m           0.435458  0.163493  0.039841  0.573705   
32     6        lms           0.368526  0.197721  0.000000  0.569721   
33     6        mel           0.364542  0.209350  0.019920  0.561753   
34     6          s           0.429880  0.180097  0.035857  0.565737   
35     7       flux           0.455777  0.150358  0.187251  0.537849   
36     7        l-m           0.502789  0.049312  0.450199  0.553785   
37     7        lms           0.502789  0.017228  0.482072  0.529880   
38     7        mel           0.358566  0.137724  0.179283  0.521912   
39     7          s           0.376096  0.091832  0.270916  0.478088   
40     8       flux           0.452988  0.147632  0.067729  0.581673   
41     8        l-m           0.398008  0.079470  0.243028  0.494024   
42     8        lms           0.480876  0.078343  0.314741  0.565737   
43     8        mel           0.434661  0.091959  0.278884  0.537849   
44     8          s           0.440637  0.084059  0.286853  0.541833   
45     9       flux           0.328685  0.199674  0.000000  0.537849   
46     9        l-m           0.320319  0.151955  0.123506  0.545817   
47     9        lms           0.357371  0.154873  0.000000  0.557769   
48     9        mel           0.361355  0.121368  0.227092  0.585657   
49     9          s           0.368924  0.141360  0.139442  0.549801   
50    10       flux           0.364143  0.217270  0.000000  0.573705   
51    10        l-m           0.343426  0.191270  0.000000  0.565737   
52    10        lms           0.392032  0.2

### Participant 206

In [106]:
print('Statistics for whole trials')
trial_stats_dict["206"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.543894  0.048693  0.412844  0.580612   
1      0        l-m           0.552026  0.032475  0.488776  0.584694   
2      0        lms           0.558053  0.015065  0.534219  0.578141   
3      0        mel           0.544670  0.023688  0.494898  0.569388   
4      0          s           0.544362  0.053585  0.418367  0.580184   
5      1       flux           0.382675  0.077408  0.335072  0.586735   
6      1        l-m           0.361508  0.080733  0.311360  0.586735   
7      1        lms           0.392411  0.102642  0.314694  0.588776   
8      1        mel           0.397347  0.129491  0.250965  0.581633   
9      1          s           0.392564  0.098404  0.333333  0.579592   
10     2       flux           0.568264  0.010603  0.550459  0.582031   
11     2        l-m           0.569416  0.009754  0.558733  0.588355   
12     2        lms           0.566970  0.026139  0.496425  0.586137   
13     2        mel           0.546648  0.086046  0.302349  0.581633   
14     2          s           0.545565  0.060323  0.394898  0.586137   
15     3       flux           0.535801  0.043775  0.477495  0.585117   
16     3        l-m           0.511073  0.090153  0.303061  0.586313   
17     3        lms           0.493921  0.122035  0.199796  0.583673   
18     3        mel           0.519245  0.061213  0.397959  0.581040   
19     3          s           0.496504  0.104297  0.255363  0.580612   
20     4       flux           0.546239  0.054100  0.404494  0.579824   
21     4        l-m           0.565433  0.017834  0.533197  0.582653   
22     4        lms           0.564172  0.013449  0.542857  0.583673   
23     4        mel           0.562805  0.023165  0.522936  0.586735   
24     4          s           0.568119  0.018509  0.535714  0.587755   
25     5       flux           0.468773  0.101345  0.338446  0.577551   
26     5        l-m           0.482524  0.076199  0.341488  0.564286   
27     5        lms           0.512070  0.048626  0.438776  0.559755   
28     5        mel           0.520522  0.044223  0.427988  0.581205   
29     5          s           0.512091  0.055208  0.417347  0.569969   
30     6       flux           0.500497  0.092215  0.274210  0.580020   
31     6        l-m           0.498832  0.057227  0.367347  0.549080   
32     6        lms           0.494664  0.072390  0.320408  0.554082   
33     6        mel           0.452065  0.077718  0.351682  0.567347   
34     6          s           0.495761  0.056498  0.400409  0.558733   
35     7       flux           0.464069  0.146433  0.183953  0.587755   
36     7        l-m           0.511646  0.122419  0.186925  0.586313   
37     7        lms           0.442451  0.188309  0.109295  0.585714   
38     7        mel           0.403971  0.184386  0.106122  0.579592   
39     7          s           0.449728  0.161691  0.213265  0.585714   
40     8       flux           0.570622  0.010672  0.549439  0.585117   
41     8        l-m           0.563225  0.016288  0.533673  0.587755   
42     8        lms           0.560345  0.020284  0.521429  0.580020   
43     8        mel           0.575294  0.008143  0.556575  0.583673   
44     8          s           0.563091  0.036294  0.464286  0.588776   
45     9       flux           0.576627  0.019499  0.532176  0.588776   
46     9        l-m           0.582713  0.006540  0.571429  0.587755   
47     9        lms           0.585226  0.003940  0.575510  0.588776   
48     9        mel           0.582673  0.008158  0.567347  0.589377   
49     9          s           0.579855  0.019145  0.525510  0.588175   
50    10       flux           0.555521  0.039572  0.477017  0.587755   
51    10        l-m           0.540082  0.056727  0.438776  0.588175   
52    10        lms           0.568301  0.0

In [107]:
print('Statistics for stimulus phase')
stim_stats_dict["206"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.564143  0.025628  0.509960  0.589641   
1      0        l-m           0.543028  0.072909  0.358566  0.593625   
2      0        lms           0.558566  0.022678  0.521912  0.589641   
3      0        mel           0.564143  0.023548  0.509960  0.589641   
4      0          s           0.539044  0.088710  0.298805  0.589641   
5      1       flux           0.119522  0.243582  0.003984  0.585657   
6      1        l-m           0.062151  0.183951  0.000000  0.585657   
7      1        lms           0.121116  0.246942  0.003984  0.593625   
8      1        mel           0.178486  0.280981  0.003984  0.589641   
9      1          s           0.119522  0.242545  0.003984  0.585657   
10     2       flux           0.573307  0.024806  0.509960  0.589641   
11     2        l-m           0.572510  0.029460  0.494024  0.589641   
12     2        lms           0.581275  0.010018  0.561753  0.589641   
13     2        mel           0.552988  0.075890  0.338645  0.585657   
14     2          s           0.564940  0.040881  0.450199  0.589641   
15     3       flux           0.526693  0.096732  0.266932  0.581673   
16     3        l-m           0.462550  0.159743  0.155378  0.589641   
17     3        lms           0.498406  0.137608  0.215139  0.585657   
18     3        mel           0.533466  0.056451  0.394422  0.581673   
19     3          s           0.464143  0.195569  0.031873  0.585657   
20     4       flux           0.523108  0.133386  0.147410  0.585657   
21     4        l-m           0.574502  0.016986  0.545817  0.589641   
22     4        lms           0.554980  0.033861  0.474104  0.581673   
23     4        mel           0.567331  0.019079  0.525896  0.589641   
24     4          s           0.569323  0.023416  0.521912  0.589641   
25     5       flux           0.496813  0.093549  0.274900  0.581673   
26     5        l-m           0.491633  0.109563  0.215139  0.577689   
27     5        lms           0.557769  0.019876  0.513944  0.581673   
28     5        mel           0.531474  0.043327  0.462151  0.589641   
29     5          s           0.499203  0.059065  0.426295  0.577689   
30     6       flux           0.513147  0.111961  0.203187  0.585657   
31     6        l-m           0.545020  0.063878  0.374502  0.589641   
32     6        lms           0.492032  0.097467  0.258964  0.577689   
33     6        mel           0.489641  0.089194  0.354582  0.589641   
34     6          s           0.511155  0.080825  0.346614  0.585657   
35     7       flux           0.449801  0.202186  0.000000  0.589641   
36     7        l-m           0.507570  0.100723  0.342629  0.589641   
37     7        lms           0.468526  0.199903  0.000000  0.585657   
38     7        mel           0.417131  0.158475  0.151394  0.581673   
39     7          s           0.501195  0.131545  0.195219  0.589641   
40     8       flux           0.562948  0.028175  0.494024  0.589641   
41     8        l-m           0.551394  0.026240  0.509960  0.589641   
42     8        lms           0.547410  0.050121  0.410359  0.581673   
43     8        mel           0.573705  0.027216  0.498008  0.589641   
44     8          s           0.557769  0.032638  0.486056  0.589641   
45     9       flux           0.557769  0.056124  0.446215  0.589641   
46     9        l-m           0.575299  0.020070  0.525896  0.589641   
47     9        lms           0.586056  0.003488  0.581673  0.593625   
48     9        mel           0.577291  0.025368  0.505976  0.589641   
49     9          s           0.584064  0.006285  0.569721  0.589641   
50    10       flux           0.573705  0.020228  0.537849  0.589641   
51    10        l-m           0.568127  0.026772  0.525896  0.589641   
52    10        lms           0.566534  0.0

### Participant 207

In [108]:
print('Statistics for whole trials')
trial_stats_dict["207"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.482174  0.110785  0.228571  0.566327   
1      0        l-m           0.536172  0.043428  0.458163  0.573469   
2      0        lms           0.493154  0.122063  0.176351  0.572885   
3      0        mel           0.491493  0.152344  0.083673  0.575510   
4      0          s           0.525567  0.076139  0.325510  0.573469   
5      1       flux           0.537935  0.012934  0.513790  0.552497   
6      1        l-m           0.545678  0.015885  0.524490  0.570408   
7      1        lms           0.537735  0.021399  0.494898  0.565749   
8      1        mel           0.526084  0.057584  0.372069  0.560652   
9      1          s           0.544495  0.013387  0.515832  0.560204   
10     2       flux           0.327102  0.156249  0.000000  0.531633   
11     2        l-m           0.447240  0.077563  0.321757  0.530133   
12     2        lms           0.412993  0.134618  0.086823  0.518897   
13     2        mel           0.421120  0.154193  0.000000  0.555669   
14     2          s           0.386396  0.123183  0.132653  0.518367   
15     3       flux           0.382867  0.126587  0.170421  0.540816   
16     3        l-m           0.358568  0.089758  0.212029  0.495413   
17     3        lms           0.402589  0.075457  0.295199  0.535168   
18     3        mel           0.379404  0.114707  0.199183  0.541837   
19     3          s           0.415291  0.086841  0.255363  0.514286   
20     4       flux           0.128713  0.100649  0.000000  0.348980   
21     4        l-m           0.190070  0.094147  0.047959  0.365953   
22     4        lms           0.214201  0.136551  0.008163  0.416327   
23     4        mel           0.176214  0.106287  0.000000  0.365953   
24     4          s           0.141180  0.131078  0.012232  0.416327   
25     5       flux           0.294774  0.104662  0.130612  0.428571   
26     5        l-m           0.305298  0.116655  0.105209  0.447395   
27     5        lms           0.252561  0.149745  0.000000  0.412245   
28     5        mel           0.308491  0.120226  0.114286  0.492857   
29     5          s           0.245908  0.137818  0.037717  0.483673   
30     6       flux           0.421920  0.140507  0.086735  0.547959   
31     6        l-m           0.426201  0.120977  0.181448  0.551020   
32     6        lms           0.375468  0.142842  0.165475  0.533197   
33     6        mel           0.381313  0.113485  0.129724  0.507143   
34     6          s           0.443452  0.105909  0.186735  0.556122   
35     7       flux           0.398756  0.139900  0.052147  0.561224   
36     7        l-m           0.447762  0.079413  0.335714  0.563265   
37     7        lms           0.419853  0.104543  0.231672  0.557712   
38     7        mel           0.399310  0.158767  0.002843  0.561798   
39     7          s           0.385982  0.182283  0.070408  0.559184   
40     8       flux           0.407734  0.139502  0.113381  0.539326   
41     8        l-m           0.372681  0.194395  0.085714  0.551583   
42     8        lms           0.422041  0.155958  0.032653  0.547959   
43     8        mel           0.379503  0.148554  0.088866  0.551020   
44     8          s           0.385334  0.176648  0.057143  0.534694   
45     9       flux           0.413319  0.176487  0.039755  0.554536   
46     9        l-m           0.511324  0.013241  0.497452  0.533673   
47     9        lms           0.411509  0.140247  0.184694  0.541837   
48     9        mel           0.471722  0.056738  0.355465  0.535714   
49     9          s           0.378539  0.198366  0.035751  0.555556   
50    10       flux           0.356618  0.097643  0.166496  0.477017   
51    10        l-m           0.390069  0.099473  0.235714  0.512768   
52    10        lms           0.286740  0.1

In [109]:
print('Statistics for stimulus phase')
stim_stats_dict["207"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.512749  0.090793  0.282869  0.585657   
1      0        l-m           0.536653  0.052823  0.438247  0.581673   
2      0        lms           0.521116  0.082526  0.342629  0.589641   
3      0        mel           0.508765  0.144175  0.111554  0.589641   
4      0          s           0.533068  0.091504  0.294821  0.585657   
5      1       flux           0.535458  0.027933  0.494024  0.573705   
6      1        l-m           0.550199  0.030943  0.505976  0.589641   
7      1        lms           0.549402  0.032554  0.490040  0.585657   
8      1        mel           0.531873  0.040160  0.466135  0.569721   
9      1          s           0.567331  0.014693  0.541833  0.589641   
10     2       flux           0.317928  0.203623  0.000000  0.541833   
11     2        l-m           0.519124  0.052522  0.426295  0.565737   
12     2        lms           0.452191  0.125444  0.191235  0.585657   
13     2        mel           0.433865  0.170458  0.000000  0.585657   
14     2          s           0.395219  0.182465  0.000000  0.561753   
15     3       flux           0.407171  0.157445  0.139442  0.589641   
16     3        l-m           0.407171  0.142610  0.107570  0.545817   
17     3        lms           0.417530  0.119917  0.223108  0.589641   
18     3        mel           0.380478  0.176694  0.000000  0.561753   
19     3          s           0.494422  0.036777  0.430279  0.533865   
20     4       flux           0.154183  0.093944  0.000000  0.266932   
21     4        l-m           0.272510  0.186484  0.000000  0.517928   
22     4        lms           0.242231  0.176570  0.000000  0.482072   
23     4        mel           0.127888  0.135776  0.000000  0.386454   
24     4          s           0.168127  0.201210  0.000000  0.589641   
25     5       flux           0.411155  0.107624  0.195219  0.553785   
26     5        l-m           0.351394  0.213227  0.063745  0.589641   
27     5        lms           0.285259  0.189039  0.000000  0.529880   
28     5        mel           0.351793  0.143329  0.031873  0.549801   
29     5          s           0.259761  0.232717  0.000000  0.561753   
30     6       flux           0.448207  0.135864  0.163347  0.565737   
31     6        l-m           0.440637  0.169552  0.047809  0.585657   
32     6        lms           0.427092  0.144514  0.207171  0.577689   
33     6        mel           0.421116  0.165327  0.123506  0.581673   
34     6          s           0.500000  0.082364  0.350598  0.561753   
35     7       flux           0.433956  0.176610  0.011952  0.585657   
36     7        l-m           0.484064  0.134733  0.075697  0.589641   
37     7        lms           0.425682  0.105532  0.262948  0.573705   
38     7        mel           0.383001  0.181707  0.000000  0.577689   
39     7          s           0.421457  0.174910  0.063745  0.589641   
40     8       flux           0.462151  0.110362  0.215139  0.589641   
41     8        l-m           0.458167  0.151033  0.235060  0.585657   
42     8        lms           0.435060  0.197682  0.000000  0.585657   
43     8        mel           0.407171  0.120372  0.151394  0.533865   
44     8          s           0.399203  0.238392  0.000000  0.589641   
45     9       flux           0.443028  0.203866  0.000000  0.589641   
46     9        l-m           0.552590  0.051539  0.422311  0.589641   
47     9        lms           0.422709  0.180479  0.000000  0.545817   
48     9        mel           0.494024  0.092467  0.274900  0.589641   
49     9          s           0.424701  0.205618  0.055777  0.585657   
50    10       flux           0.373705  0.163350  0.079681  0.569721   
51    10        l-m           0.441833  0.110784  0.250996  0.569721   
52    10        lms           0.308765  0.1

### Participant 209

In [110]:
print('Statistics for whole trials')
trial_stats_dict["209"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.221087  0.103753  0.009184  0.376915   
1      0        l-m           0.201019  0.147234  0.000000  0.458163   
2      0        lms           0.181820  0.143837  0.000000  0.383044   
3      0        mel           0.236619  0.147836  0.017365  0.469388   
4      0          s           0.195369  0.153918  0.000000  0.418534   
5      1       flux           0.238458  0.160376  0.000000  0.576531   
6      1        l-m           0.223056  0.164333  0.042857  0.486735   
7      1        lms           0.260162  0.139297  0.000000  0.530071   
8      1        mel           0.288901  0.174226  0.000000  0.588355   
9      1          s           0.211563  0.169855  0.000000  0.587334   
10     2       flux           0.343203  0.144818  0.090816  0.566536   
11     2        l-m           0.379838  0.162783  0.181818  0.576098   
12     2        lms           0.312289  0.172741  0.044898  0.557712   
13     2        mel           0.347851  0.194889  0.040816  0.564286   
14     2          s           0.317398  0.164323  0.026558  0.541369   
15     3       flux           0.204926  0.177041  0.012270  0.451943   
16     3        l-m           0.129258  0.122371  0.000000  0.400000   
17     3        lms           0.192075  0.178857  0.000000  0.440816   
18     3        mel           0.224586  0.201990  0.000000  0.548057   
19     3          s           0.175141  0.135311  0.029592  0.461224   
20     4       flux           0.268422  0.153478  0.000000  0.517347   
21     4        l-m           0.238856  0.119246  0.033673  0.420570   
22     4        lms           0.285490  0.197142  0.015306  0.580184   
23     4        mel           0.269864  0.174859  0.000000  0.534694   
24     4          s           0.290824  0.179859  0.000000  0.523469   
25     5       flux           0.505234  0.055056  0.387755  0.552838   
26     5        l-m           0.522656  0.028254  0.456033  0.553061   
27     5        lms           0.518806  0.039222  0.441267  0.567347   
28     5        mel           0.513835  0.039748  0.424490  0.552605   
29     5          s           0.486471  0.052168  0.386108  0.533197   
30     6       flux           0.071367  0.098485  0.000000  0.281920   
31     6        l-m           0.037669  0.059717  0.000000  0.195918   
32     6        lms           0.043616  0.062657  0.000000  0.167689   
33     6        mel           0.105877  0.157335  0.000000  0.409786   
34     6          s           0.063750  0.097885  0.000000  0.250765   
35     7       flux           0.033302  0.083719  0.000000  0.267620   
36     7        l-m           0.009795  0.018774  0.000000  0.048980   
37     7        lms           0.025102  0.074496  0.000000  0.236735   
38     7        mel           0.007037  0.011822  0.000000  0.031633   
39     7          s           0.047245  0.063642  0.000000  0.182840   
40     8       flux           0.358678  0.177887  0.165306  0.585117   
41     8        l-m           0.431452  0.125310  0.216327  0.572012   
42     8        lms           0.411542  0.137604  0.190622  0.572449   
43     8        mel           0.417058  0.094478  0.251020  0.580612   
44     8          s           0.395971  0.181673  0.080612  0.567347   
45     9       flux           0.323920  0.122727  0.116327  0.475996   
46     9        l-m           0.308668  0.128453  0.080612  0.476483   
47     9        lms           0.396765  0.100634  0.258317  0.542304   
48     9        mel           0.369113  0.164309  0.009193  0.559184   
49     9          s           0.415897  0.065525  0.318043  0.538776   
50    10       flux           0.388007  0.142512  0.161224  0.534694   
51    10        l-m           0.381885  0.114727  0.195097  0.487755   
52    10        lms           0.341591  0.1

In [111]:
print('Statistics for stimulus phase')
stim_stats_dict["209"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.275697  0.172651  0.015936  0.577689   
1      0        l-m           0.206773  0.181395  0.000000  0.482072   
2      0        lms           0.117928  0.131134  0.000000  0.342629   
3      0        mel           0.206375  0.138481  0.003984  0.442231   
4      0          s           0.175697  0.199914  0.000000  0.529880   
5      1       flux           0.232669  0.174825  0.000000  0.569721   
6      1        l-m           0.233997  0.188978  0.000000  0.545817   
7      1        lms           0.270916  0.184783  0.000000  0.541833   
8      1        mel           0.279947  0.214524  0.000000  0.593625   
9      1          s           0.242497  0.208077  0.000000  0.585657   
10     2       flux           0.333865  0.157210  0.103586  0.573705   
11     2        l-m           0.378884  0.172688  0.107570  0.581673   
12     2        lms           0.309960  0.205511  0.000000  0.533865   
13     2        mel           0.321912  0.205503  0.000000  0.561753   
14     2          s           0.331873  0.176232  0.055777  0.589641   
15     3       flux           0.199602  0.203369  0.000000  0.541833   
16     3        l-m           0.098008  0.171938  0.000000  0.525896   
17     3        lms           0.172510  0.187218  0.000000  0.498008   
18     3        mel           0.266135  0.207782  0.000000  0.565737   
19     3          s           0.183665  0.161194  0.000000  0.406375   
20     4       flux           0.238247  0.252043  0.000000  0.569721   
21     4        l-m           0.263745  0.190892  0.000000  0.513944   
22     4        lms           0.288446  0.216839  0.000000  0.589641   
23     4        mel           0.232271  0.178509  0.000000  0.569721   
24     4          s           0.224701  0.193100  0.000000  0.553785   
25     5       flux           0.489243  0.076929  0.322709  0.589641   
26     5        l-m           0.537450  0.030599  0.462151  0.561753   
27     5        lms           0.521912  0.038029  0.466135  0.573705   
28     5        mel           0.508765  0.068828  0.330677  0.573705   
29     5          s           0.485657  0.082411  0.330677  0.585657   
30     6       flux           0.027490  0.050725  0.000000  0.151394   
31     6        l-m           0.030279  0.056599  0.000000  0.147410   
32     6        lms           0.053386  0.109499  0.000000  0.314741   
33     6        mel           0.090040  0.148135  0.000000  0.454183   
34     6          s           0.081673  0.135422  0.000000  0.386454   
35     7       flux           0.030677  0.059629  0.000000  0.183267   
36     7        l-m           0.012749  0.040316  0.000000  0.127490   
37     7        lms           0.013546  0.042836  0.000000  0.135458   
38     7        mel           0.004781  0.015118  0.000000  0.047809   
39     7          s           0.035458  0.095330  0.000000  0.302789   
40     8       flux           0.374502  0.215557  0.031873  0.593625   
41     8        l-m           0.428685  0.189923  0.031873  0.581673   
42     8        lms           0.424303  0.176794  0.139442  0.589641   
43     8        mel           0.428685  0.136718  0.227092  0.581673   
44     8          s           0.393625  0.191279  0.000000  0.569721   
45     9       flux           0.295618  0.101623  0.151394  0.462151   
46     9        l-m           0.338645  0.176762  0.083665  0.573705   
47     9        lms           0.378088  0.165736  0.055777  0.557769   
48     9        mel           0.408765  0.229914  0.000000  0.581673   
49     9          s           0.444622  0.149096  0.067729  0.577689   
50    10       flux           0.389641  0.147306  0.107570  0.569721   
51    10        l-m           0.323904  0.134348  0.063745  0.486056   
52    10        lms           0.303984  0.2

### Participant 210

In [112]:
print('Statistics for whole trials')
trial_stats_dict["210"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.493834  0.044927  0.408163  0.555669   
1      0        l-m           0.473511  0.074685  0.342186  0.553626   
2      0        lms           0.494385  0.067865  0.372829  0.580645   
3      0        mel           0.476460  0.063800  0.368744  0.558733   
4      0          s           0.499944  0.044910  0.429592  0.576531   
5      1       flux           0.324656  0.138124  0.080695  0.481409   
6      1        l-m           0.304096  0.154293  0.020387  0.569827   
7      1        lms           0.343425  0.112757  0.158325  0.547959   
8      1        mel           0.310111  0.103183  0.158163  0.458716   
9      1          s           0.387319  0.073490  0.275229  0.517347   
10     2       flux           0.458377  0.077985  0.284694  0.547959   
11     2        l-m           0.464124  0.089853  0.292857  0.551583   
12     2        lms           0.477832  0.059154  0.356513  0.541284   
13     2        mel           0.438267  0.090487  0.295918  0.527068   
14     2          s           0.467041  0.061043  0.365953  0.547959   
15     3       flux           0.344674  0.174820  0.104082  0.547959   
16     3        l-m           0.326208  0.148085  0.127681  0.550459   
17     3        lms           0.299630  0.200409  0.030333  0.562245   
18     3        mel           0.326624  0.194589  0.047910  0.543878   
19     3          s           0.317217  0.154057  0.087755  0.510204   
20     4       flux           0.288219  0.187299  0.051935  0.563841   
21     4        l-m           0.363789  0.113842  0.146939  0.479103   
22     4        lms           0.260341  0.111943  0.126402  0.478571   
23     4        mel           0.348548  0.121313  0.138776  0.478039   
24     4          s           0.257944  0.114732  0.127291  0.483673   
25     5       flux           0.296582  0.147087  0.138776  0.551020   
26     5        l-m           0.409795  0.143515  0.192865  0.582227   
27     5        lms           0.251195  0.170530  0.026558  0.523955   
28     5        mel           0.288744  0.142783  0.022449  0.487755   
29     5          s           0.346353  0.158055  0.030822  0.565306   
30     6       flux           0.312075  0.155050  0.101020  0.522449   
31     6        l-m           0.396454  0.118132  0.159346  0.531633   
32     6        lms           0.393776  0.156888  0.107143  0.525510   
33     6        mel           0.308566  0.129116  0.138776  0.498471   
34     6          s           0.351877  0.140806  0.131498  0.512742   
35     7       flux           0.227854  0.091827  0.113265  0.383187   
36     7        l-m           0.277022  0.082650  0.133810  0.407747   
37     7        lms           0.326425  0.127981  0.103166  0.496939   
38     7        mel           0.283622  0.153928  0.090909  0.540816   
39     7          s           0.256323  0.123834  0.101020  0.480612   
40     8       flux           0.291753  0.099484  0.174668  0.468846   
41     8        l-m           0.286146  0.113684  0.128571  0.504082   
42     8        lms           0.268318  0.141665  0.000000  0.450000   
43     8        mel           0.229173  0.116162  0.042857  0.380224   
44     8          s           0.244960  0.139499  0.128571  0.575510   
45     9       flux           0.296155  0.150342  0.093878  0.489796   
46     9        l-m           0.437840  0.067330  0.313265  0.560776   
47     9        lms           0.284695  0.122708  0.141837  0.443878   
48     9        mel           0.318047  0.130939  0.164286  0.536188   
49     9          s           0.366419  0.082937  0.220408  0.521472   
50    10       flux           0.239098  0.173110  0.017329  0.525994   
51    10        l-m           0.336685  0.122662  0.161060  0.513265   
52    10        lms           0.179664  0.1

In [113]:
print('Statistics for stimulus phase')
stim_stats_dict["210"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.522311  0.085437  0.322709  0.589641   
1      0        l-m           0.485259  0.062679  0.370518  0.557769   
2      0        lms           0.533466  0.057595  0.394422  0.589641   
3      0        mel           0.505578  0.054512  0.378486  0.565737   
4      0          s           0.489243  0.056587  0.362550  0.561753   
5      1       flux           0.321514  0.216032  0.000000  0.533865   
6      1        l-m           0.347410  0.211058  0.000000  0.581673   
7      1        lms           0.331076  0.151772  0.091633  0.482072   
8      1        mel           0.386853  0.156328  0.063745  0.573705   
9      1          s           0.412749  0.146939  0.167331  0.585657   
10     2       flux           0.458167  0.109334  0.219124  0.537849   
11     2        l-m           0.537450  0.052232  0.438247  0.589641   
12     2        lms           0.516335  0.057127  0.378486  0.577689   
13     2        mel           0.449801  0.084878  0.282869  0.545817   
14     2          s           0.506375  0.036148  0.426295  0.549801   
15     3       flux           0.393227  0.149920  0.199203  0.581673   
16     3        l-m           0.469323  0.116500  0.179283  0.573705   
17     3        lms           0.366135  0.210988  0.000000  0.581673   
18     3        mel           0.423506  0.161408  0.187251  0.589641   
19     3          s           0.342629  0.132975  0.123506  0.545817   
20     4       flux           0.303187  0.143935  0.079681  0.533865   
21     4        l-m           0.427092  0.109717  0.183267  0.549801   
22     4        lms           0.352590  0.219540  0.000000  0.585657   
23     4        mel           0.317530  0.134950  0.000000  0.505976   
24     4          s           0.339044  0.132755  0.007968  0.482072   
25     5       flux           0.301195  0.198184  0.059761  0.577689   
26     5        l-m           0.398406  0.165038  0.195219  0.593625   
27     5        lms           0.303984  0.233792  0.000000  0.585657   
28     5        mel           0.301195  0.220933  0.000000  0.589641   
29     5          s           0.335857  0.223329  0.003984  0.585657   
30     6       flux           0.338645  0.138331  0.083665  0.557769   
31     6        l-m           0.508765  0.056313  0.434263  0.589641   
32     6        lms           0.472112  0.137156  0.171315  0.589641   
33     6        mel           0.441434  0.074198  0.350598  0.589641   
34     6          s           0.412749  0.096265  0.254980  0.569721   
35     7       flux           0.276096  0.143648  0.031873  0.462151   
36     7        l-m           0.386853  0.104424  0.227092  0.549801   
37     7        lms           0.339841  0.147321  0.111554  0.521912   
38     7        mel           0.368526  0.148123  0.119522  0.573705   
39     7          s           0.386454  0.104027  0.247012  0.529880   
40     8       flux           0.290837  0.181293  0.000000  0.573705   
41     8        l-m           0.441833  0.101686  0.274900  0.585657   
42     8        lms           0.356574  0.182159  0.000000  0.557769   
43     8        mel           0.265737  0.190005  0.000000  0.569721   
44     8          s           0.331474  0.143522  0.099602  0.581673   
45     9       flux           0.356175  0.182855  0.079681  0.573705   
46     9        l-m           0.521514  0.054995  0.446215  0.589641   
47     9        lms           0.378088  0.183050  0.000000  0.569721   
48     9        mel           0.412749  0.128389  0.143426  0.565737   
49     9          s           0.477291  0.109460  0.338645  0.589641   
50    10       flux           0.318725  0.227801  0.000000  0.581673   
51    10        l-m           0.475697  0.096210  0.298805  0.589641   
52    10        lms           0.167331  0.1

### Participant 211

In [114]:
print('Statistics for whole trials')
trial_stats_dict["211"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.540020  0.029194  0.496939  0.588776   
1      0        l-m           0.561674  0.014540  0.541369  0.580612   
2      0        lms           0.557013  0.026686  0.494129  0.580612   
3      0        mel           0.534857  0.048196  0.420838  0.586735   
4      0          s           0.518464  0.035170  0.459653  0.555102   
5      1       flux           0.410159  0.179134  0.176711  0.584555   
6      1        l-m           0.482322  0.110446  0.250000  0.575077   
7      1        lms           0.459043  0.121048  0.257900  0.577982   
8      1        mel           0.451669  0.156856  0.205912  0.586313   
9      1          s           0.495152  0.092551  0.307143  0.581633   
10     2       flux           0.549606  0.034468  0.489796  0.581205   
11     2        l-m           0.565388  0.012084  0.543412  0.577120   
12     2        lms           0.545629  0.026065  0.497446  0.581040   
13     2        mel           0.562547  0.020098  0.523469  0.588175   
14     2          s           0.544841  0.029502  0.489275  0.581040   
15     3       flux           0.564803  0.028449  0.491892  0.588776   
16     3        l-m           0.565448  0.016288  0.536735  0.585291   
17     3        lms           0.549711  0.027904  0.499490  0.578571   
18     3        mel           0.559190  0.017169  0.533673  0.577551   
19     3          s           0.560230  0.030194  0.492355  0.583078   
20     4       flux           0.522337  0.058990  0.375894  0.570552   
21     4        l-m           0.474391  0.177081  0.071356  0.576531   
22     4        lms           0.545223  0.034334  0.470408  0.582059   
23     4        mel           0.464966  0.161466  0.085714  0.580184   
24     4          s           0.521728  0.088049  0.285714  0.583248   
25     5       flux           0.506609  0.054911  0.427115  0.573469   
26     5        l-m           0.486809  0.065170  0.357873  0.566327   
27     5        lms           0.528250  0.048664  0.467824  0.581372   
28     5        mel           0.499807  0.070044  0.364286  0.573034   
29     5          s           0.492355  0.075630  0.358818  0.573904   
30     6       flux           0.467302  0.063560  0.341837  0.539796   
31     6        l-m           0.424286  0.110009  0.308163  0.563265   
32     6        lms           0.480147  0.081890  0.300306  0.572012   
33     6        mel           0.448525  0.085400  0.355759  0.584694   
34     6          s           0.501902  0.076599  0.360204  0.577551   
35     7       flux           0.491861  0.094434  0.278571  0.582227   
36     7        l-m           0.459988  0.133116  0.238776  0.585117   
37     7        lms           0.504238  0.050383  0.436159  0.579162   
38     7        mel           0.529855  0.031993  0.490816  0.580612   
39     7          s           0.474463  0.099814  0.293878  0.586313   
40     8       flux           0.553857  0.025115  0.501018  0.574619   
41     8        l-m           0.555954  0.046433  0.434694  0.586313   
42     8        lms           0.562871  0.024235  0.507645  0.583248   
43     8        mel           0.549785  0.038722  0.467890  0.583078   
44     8          s           0.553728  0.026312  0.494393  0.578571   
45     9       flux           0.581011  0.013715  0.543878  0.588355   
46     9        l-m           0.575515  0.016280  0.539715  0.587755   
47     9        lms           0.578486  0.009082  0.566769  0.587755   
48     9        mel           0.574336  0.020957  0.528090  0.589377   
49     9          s           0.579609  0.007217  0.567347  0.588355   
50    10       flux           0.567471  0.019470  0.519388  0.587334   
51    10        l-m           0.577114  0.007071  0.569388  0.588355   
52    10        lms           0.571491  0.0

In [115]:
print('Statistics for stimulus phase')
stim_stats_dict["211"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.534661  0.045418  0.466135  0.589641   
1      0        l-m           0.573307  0.018445  0.533865  0.589641   
2      0        lms           0.578088  0.017157  0.533865  0.589641   
3      0        mel           0.563347  0.035396  0.501992  0.589641   
4      0          s           0.543028  0.032695  0.478088  0.577689   
5      1       flux           0.440239  0.199037  0.035857  0.589641   
6      1        l-m           0.493227  0.140617  0.175299  0.589641   
7      1        lms           0.455378  0.197782  0.000000  0.589641   
8      1        mel           0.402390  0.229321  0.039841  0.585657   
9      1          s           0.532271  0.080215  0.382470  0.589641   
10     2       flux           0.570916  0.023908  0.509960  0.589641   
11     2        l-m           0.580080  0.009239  0.565737  0.589641   
12     2        lms           0.568526  0.027542  0.509960  0.589641   
13     2        mel           0.571713  0.019720  0.533865  0.589641   
14     2          s           0.547809  0.078752  0.338645  0.589641   
15     3       flux           0.581275  0.009840  0.557769  0.589641   
16     3        l-m           0.577689  0.015258  0.541833  0.589641   
17     3        lms           0.558964  0.030920  0.509960  0.589641   
18     3        mel           0.568924  0.021561  0.521912  0.593625   
19     3          s           0.551793  0.073035  0.346614  0.585657   
20     4       flux           0.549004  0.047207  0.438247  0.589641   
21     4        l-m           0.496414  0.178373  0.031873  0.589641   
22     4        lms           0.560159  0.047371  0.434263  0.593625   
23     4        mel           0.403984  0.206900  0.000000  0.589641   
24     4          s           0.512749  0.161058  0.087649  0.589641   
25     5       flux           0.470916  0.097946  0.314741  0.589641   
26     5        l-m           0.480876  0.118515  0.282869  0.585657   
27     5        lms           0.533068  0.061141  0.422311  0.585657   
28     5        mel           0.495618  0.107843  0.235060  0.585657   
29     5          s           0.513147  0.069001  0.334661  0.581673   
30     6       flux           0.526295  0.083305  0.338645  0.593625   
31     6        l-m           0.447012  0.192950  0.000000  0.589641   
32     6        lms           0.496016  0.089840  0.306773  0.577689   
33     6        mel           0.467729  0.143318  0.171315  0.589641   
34     6          s           0.503187  0.124467  0.270916  0.589641   
35     7       flux           0.443426  0.172664  0.031873  0.589641   
36     7        l-m           0.420717  0.215215  0.000000  0.589641   
37     7        lms           0.450996  0.140830  0.171315  0.589641   
38     7        mel           0.544622  0.064898  0.394422  0.589641   
39     7          s           0.458964  0.127653  0.239044  0.585657   
40     8       flux           0.562948  0.018406  0.517928  0.589641   
41     8        l-m           0.558566  0.042984  0.442231  0.585657   
42     8        lms           0.544622  0.068828  0.354582  0.585657   
43     8        mel           0.545817  0.045425  0.466135  0.589641   
44     8          s           0.534661  0.080972  0.322709  0.577689   
45     9       flux           0.588048  0.002057  0.585657  0.589641   
46     9        l-m           0.580876  0.008968  0.561753  0.589641   
47     9        lms           0.584462  0.007040  0.569721  0.589641   
48     9        mel           0.554183  0.073425  0.394422  0.589641   
49     9          s           0.583267  0.013439  0.545817  0.589641   
50    10       flux           0.562151  0.028634  0.509960  0.589641   
51    10        l-m           0.588845  0.001680  0.585657  0.589641   
52    10        lms           0.574900  0.0

### Participant 212

In [116]:
print('Statistics for whole trials')
trial_stats_dict["212"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.465358  0.170293  0.139796  0.588776   
1      0        l-m           0.497236  0.135597  0.226531  0.588776   
2      0        lms           0.432850  0.145903  0.152905  0.585714   
3      0        mel           0.423180  0.166166  0.098059  0.587755   
4      0          s           0.456728  0.161136  0.144898  0.587755   
5      1       flux           0.401213  0.162715  0.095918  0.588776   
6      1        l-m           0.376368  0.192010  0.035714  0.588175   
7      1        lms           0.397758  0.202895  0.043922  0.588776   
8      1        mel           0.372205  0.207443  0.033639  0.586735   
9      1          s           0.370447  0.245114  0.000000  0.587755   
10     2       flux           0.498617  0.185058  0.000000  0.587755   
11     2        l-m           0.558489  0.066332  0.374490  0.588776   
12     2        lms           0.514350  0.175057  0.020492  0.587755   
13     2        mel           0.516570  0.182272  0.000000  0.589195   
14     2          s           0.422828  0.241270  0.000000  0.588355   
15     3       flux           0.586907  0.001621  0.583248  0.588355   
16     3        l-m           0.573512  0.023555  0.520408  0.588776   
17     3        lms           0.575117  0.021387  0.532587  0.588355   
18     3        mel           0.586345  0.002419  0.580020  0.588776   
19     3          s           0.584922  0.002968  0.580777  0.588355   
20     4       flux           0.561490  0.045695  0.467347  0.588063   
21     4        l-m           0.583180  0.006571  0.568367  0.589377   
22     4        lms           0.585744  0.003078  0.579162  0.588776   
23     4        mel           0.583852  0.004045  0.576531  0.588175   
24     4          s           0.578835  0.020337  0.521429  0.587755   
25     5       flux           0.578989  0.011237  0.555102  0.588776   
26     5        l-m           0.578249  0.007316  0.563710  0.587156   
27     5        lms           0.582317  0.006057  0.568948  0.588355   
28     5        mel           0.583752  0.005522  0.572449  0.589377   
29     5          s           0.569792  0.019325  0.528571  0.588776   
30     6       flux           0.435284  0.131652  0.140059  0.587755   
31     6        l-m           0.334064  0.183277  0.101020  0.550562   
32     6        lms           0.463160  0.104663  0.291539  0.587334   
33     6        mel           0.446425  0.132501  0.198980  0.585714   
34     6          s           0.491317  0.091162  0.310521  0.586735   
35     7       flux           0.551814  0.102189  0.261224  0.588776   
36     7        l-m           0.529222  0.086067  0.364286  0.588776   
37     7        lms           0.584325  0.002567  0.579592  0.586735   
38     7        mel           0.548237  0.067779  0.392638  0.588355   
39     7          s           0.578373  0.020189  0.522449  0.588776   
40     8       flux           0.581835  0.011328  0.555669  0.588776   
41     8        l-m           0.554312  0.055938  0.414286  0.587755   
42     8        lms           0.584629  0.003444  0.578571  0.587755   
43     8        mel           0.583573  0.007379  0.563710  0.588175   
44     8          s           0.580504  0.020279  0.522983  0.588355   
45     9       flux           0.560729  0.047740  0.431633  0.586313   
46     9        l-m           0.492137  0.137168  0.256881  0.586137   
47     9        lms           0.555530  0.063552  0.389173  0.586912   
48     9        mel           0.561401  0.052741  0.414286  0.588355   
49     9          s           0.572372  0.020684  0.518367  0.585714   
50    10       flux           0.560638  0.042952  0.450000  0.587084   
51    10        l-m           0.407519  0.240504  0.018443  0.588957   
52    10        lms           0.533763  0.1

In [117]:
print('Statistics for stimulus phase')
stim_stats_dict["212"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.528287  0.096155  0.342629  0.593625   
1      0        l-m           0.464940  0.207018  0.000000  0.589641   
2      0        lms           0.450598  0.183857  0.151394  0.589641   
3      0        mel           0.430677  0.237181  0.000000  0.585657   
4      0          s           0.451394  0.230442  0.000000  0.589641   
5      1       flux           0.397610  0.238296  0.000000  0.589641   
6      1        l-m           0.323904  0.261489  0.000000  0.589641   
7      1        lms           0.398805  0.222866  0.055777  0.589641   
8      1        mel           0.398805  0.229632  0.000000  0.589641   
9      1          s           0.356574  0.261511  0.000000  0.589641   
10     2       flux           0.481673  0.200627  0.000000  0.589641   
11     2        l-m           0.554183  0.067074  0.394422  0.589641   
12     2        lms           0.500797  0.189186  0.035857  0.593625   
13     2        mel           0.519920  0.185298  0.000000  0.589641   
14     2          s           0.457371  0.242442  0.000000  0.593625   
15     3       flux           0.587649  0.002100  0.585657  0.589641   
16     3        l-m           0.559363  0.069698  0.366534  0.589641   
17     3        lms           0.580478  0.016592  0.533865  0.589641   
18     3        mel           0.587251  0.002786  0.581673  0.589641   
19     3          s           0.586454  0.003661  0.581673  0.589641   
20     4       flux           0.516733  0.131226  0.247012  0.593625   
21     4        l-m           0.586454  0.004115  0.577689  0.589641   
22     4        lms           0.588845  0.001680  0.585657  0.589641   
23     4        mel           0.585657  0.003253  0.577689  0.589641   
24     4          s           0.587649  0.002817  0.581673  0.589641   
25     5       flux           0.580478  0.011736  0.557769  0.589641   
26     5        l-m           0.585259  0.005126  0.573705  0.589641   
27     5        lms           0.583665  0.007800  0.569721  0.589641   
28     5        mel           0.587649  0.004695  0.577689  0.593625   
29     5          s           0.573307  0.030714  0.505976  0.589641   
30     6       flux           0.380478  0.179015  0.107570  0.589641   
31     6        l-m           0.280080  0.220476  0.000000  0.565737   
32     6        lms           0.452988  0.196879  0.000000  0.589641   
33     6        mel           0.496813  0.123984  0.286853  0.593625   
34     6          s           0.524701  0.091316  0.298805  0.589641   
35     7       flux           0.541434  0.135724  0.155378  0.593625   
36     7        l-m           0.518327  0.116495  0.266932  0.589641   
37     7        lms           0.584462  0.005954  0.569721  0.589641   
38     7        mel           0.536255  0.091935  0.354582  0.589641   
39     7          s           0.570120  0.031340  0.490040  0.589641   
40     8       flux           0.581275  0.021203  0.521912  0.593625   
41     8        l-m           0.498406  0.142147  0.187251  0.589641   
42     8        lms           0.588048  0.002057  0.585657  0.589641   
43     8        mel           0.587649  0.003872  0.581673  0.593625   
44     8          s           0.572908  0.044634  0.446215  0.589641   
45     9       flux           0.546614  0.104193  0.250996  0.589641   
46     9        l-m           0.480080  0.183250  0.067729  0.589641   
47     9        lms           0.568924  0.030562  0.486056  0.585657   
48     9        mel           0.541434  0.124756  0.187251  0.589641   
49     9          s           0.564940  0.047431  0.430279  0.585657   
50    10       flux           0.518725  0.133197  0.175299  0.589641   
51    10        l-m           0.385259  0.262142  0.000000  0.589641   
52    10        lms           0.537849  0.1

### Participant 213

In [118]:
print('Statistics for whole trials')
trial_stats_dict["213"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.394958  0.164286  0.000000  0.586735   
1      0        l-m           0.427859  0.216280  0.000000  0.581205   
2      0        lms           0.294717  0.228111  0.000000  0.544343   
3      0        mel           0.308478  0.267565  0.000000  0.553626   
4      0          s           0.358891  0.207142  0.039837  0.587755   
5      1       flux           0.573166  0.014302  0.548519  0.588776   
6      1        l-m           0.521177  0.066643  0.375510  0.584270   
7      1        lms           0.556398  0.039936  0.457696  0.587755   
8      1        mel           0.564490  0.020926  0.523955  0.587755   
9      1          s           0.546438  0.086355  0.307143  0.588355   
10     2       flux           0.538914  0.088767  0.358529  0.589377   
11     2        l-m           0.549537  0.048573  0.444898  0.587334   
12     2        lms           0.547768  0.102487  0.259184  0.588776   
13     2        mel           0.479022  0.181698  0.113265  0.587755   
14     2          s           0.531725  0.104765  0.263534  0.588776   
15     3       flux           0.540664  0.082332  0.315306  0.588465   
16     3        l-m           0.477357  0.074696  0.352041  0.581633   
17     3        lms           0.512539  0.072811  0.378571  0.587755   
18     3        mel           0.531045  0.099031  0.267347  0.588355   
19     3          s           0.490395  0.129096  0.197757  0.588175   
20     4       flux           0.538781  0.077039  0.338776  0.584694   
21     4        l-m           0.531208  0.091173  0.297242  0.584694   
22     4        lms           0.520400  0.090544  0.323829  0.588776   
23     4        mel           0.542236  0.050875  0.429009  0.583673   
24     4          s           0.557197  0.045789  0.447395  0.587334   
25     5       flux           0.422249  0.137966  0.166496  0.587755   
26     5        l-m           0.415266  0.154994  0.169388  0.580448   
27     5        lms           0.427414  0.123908  0.231397  0.586735   
28     5        mel           0.407032  0.170721  0.036735  0.573469   
29     5          s           0.374133  0.206123  0.040858  0.566769   
30     6       flux           0.446868  0.147713  0.114286  0.576098   
31     6        l-m           0.306524  0.172731  0.045918  0.584694   
32     6        lms           0.398185  0.138736  0.185714  0.542304   
33     6        mel           0.361063  0.172855  0.085802  0.587334   
34     6          s           0.294056  0.201102  0.020429  0.588776   
35     7       flux           0.397976  0.159342  0.074414  0.588776   
36     7        l-m           0.307473  0.172396  0.059184  0.529592   
37     7        lms           0.328277  0.192345  0.029592  0.566769   
38     7        mel           0.249641  0.157123  0.072523  0.523469   
39     7          s           0.418433  0.091150  0.290816  0.546939   
40     8       flux           0.305391  0.238390  0.014300  0.588776   
41     8        l-m           0.240460  0.201419  0.000000  0.574490   
42     8        lms           0.281936  0.229727  0.009184  0.586735   
43     8        mel           0.250937  0.212454  0.012257  0.587334   
44     8          s           0.322946  0.246647  0.016327  0.588776   
45     9       flux           0.377430  0.153590  0.122449  0.574490   
46     9        l-m           0.387325  0.194240  0.110317  0.587334   
47     9        lms           0.494385  0.122888  0.165306  0.588776   
48     9        mel           0.486947  0.141356  0.193054  0.585714   
49     9          s           0.472377  0.127200  0.224490  0.586735   
50    10       flux           0.511241  0.097277  0.291837  0.588063   
51    10        l-m           0.456704  0.075164  0.337079  0.586735   
52    10        lms           0.541410  0.1

In [119]:
print('Statistics for stimulus phase')
stim_stats_dict["213"]

Statistics for stimulus phase


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.400797  0.192734  0.000000  0.589641   
1      0        l-m           0.415936  0.207428  0.000000  0.589641   
2      0        lms           0.298008  0.268666  0.000000  0.589641   
3      0        mel           0.295219  0.267526  0.000000  0.589641   
4      0          s           0.408367  0.242968  0.000000  0.593625   
5      1       flux           0.575299  0.022475  0.525896  0.593625   
6      1        l-m           0.516733  0.108184  0.286853  0.589641   
7      1        lms           0.580478  0.014186  0.541833  0.589641   
8      1        mel           0.540637  0.083212  0.338645  0.589641   
9      1          s           0.581673  0.011729  0.553785  0.589641   
10     2       flux           0.537052  0.116409  0.219124  0.593625   
11     2        l-m           0.506773  0.126877  0.223108  0.589641   
12     2        lms           0.563347  0.054407  0.418327  0.589641   
13     2        mel           0.431076  0.220106  0.000000  0.593625   
14     2          s           0.475299  0.179249  0.183267  0.589641   
15     3       flux           0.522311  0.137300  0.147410  0.589641   
16     3        l-m           0.355777  0.155532  0.139442  0.581673   
17     3        lms           0.447012  0.151369  0.167331  0.589641   
18     3        mel           0.543825  0.065364  0.406375  0.593625   
19     3          s           0.518725  0.104126  0.286853  0.589641   
20     4       flux           0.536653  0.091355  0.310757  0.589641   
21     4        l-m           0.517530  0.121852  0.219124  0.589641   
22     4        lms           0.475697  0.147119  0.199203  0.589641   
23     4        mel           0.544622  0.080234  0.354582  0.589641   
24     4          s           0.556574  0.090754  0.298805  0.593625   
25     5       flux           0.461753  0.176796  0.067729  0.589641   
26     5        l-m           0.443028  0.149717  0.127490  0.581673   
27     5        lms           0.469323  0.142226  0.203187  0.589641   
28     5        mel           0.365737  0.171934  0.027888  0.593625   
29     5          s           0.357769  0.242114  0.000000  0.589641   
30     6       flux           0.418327  0.208207  0.000000  0.589641   
31     6        l-m           0.298008  0.213409  0.000000  0.581673   
32     6        lms           0.350996  0.180879  0.051793  0.585657   
33     6        mel           0.323108  0.245540  0.043825  0.589641   
34     6          s           0.278884  0.235685  0.000000  0.593625   
35     7       flux           0.402390  0.197728  0.047809  0.589641   
36     7        l-m           0.358566  0.187180  0.119522  0.569721   
37     7        lms           0.349801  0.243037  0.000000  0.589641   
38     7        mel           0.282072  0.208781  0.000000  0.545817   
39     7          s           0.387649  0.203980  0.047809  0.589641   
40     8       flux           0.326295  0.237084  0.000000  0.593625   
41     8        l-m           0.219920  0.214415  0.000000  0.589641   
42     8        lms           0.291633  0.204376  0.000000  0.589641   
43     8        mel           0.204781  0.243635  0.000000  0.589641   
44     8          s           0.310757  0.238830  0.000000  0.589641   
45     9       flux           0.384064  0.158944  0.079681  0.589641   
46     9        l-m           0.379681  0.236873  0.063745  0.589641   
47     9        lms           0.458167  0.146793  0.111554  0.589641   
48     9        mel           0.519920  0.114476  0.219124  0.593625   
49     9          s           0.451394  0.157471  0.151394  0.589641   
50    10       flux           0.500398  0.089110  0.358566  0.589641   
51    10        l-m           0.347809  0.142403  0.159363  0.581673   
52    10        lms           0.537849  0.1

## Analysis of data resampled after removing rows where the other eye was being recorded

Motivation: around 50% of data should be missing by default, since the recording was done in an alternating manner (left-right-left-right). Therefore, it may be prudent to remove the rows which have NaN in stimulated eye but not NaN in non-stimulated eye. It shouldn't change a lot since the resampling method ignores NaN when averaging in bins, however there might be slight differences. This way we also make use of information on which NaNs are not a part of the signal.

In [120]:
data_dir = "./results/new/removed_resampled/"
trial_stats_dict = {}
stim_stats_dict = {}
for participant_id in participant_list:
    data_path = os.path.join(data_dir, str(participant_id) + "_removed_resampled_data.csv")
    data_df_new = pd.read_csv(data_path)

    groupby_df = (
        data_df_new[
            ["Block", "Trial no", "Trial type", "Trial phase", "Stim eye - Size Mm"]
        ]
        .groupby(["Block", "Trial no", "Trial type"])
        .agg(["count", "size"])
        .reset_index()
    )
    groupby_df[("Stim eye - Size Mm", "count/size ratio")] = (
        groupby_df[("Stim eye - Size Mm", "count")]
        / groupby_df[("Stim eye - Size Mm", "size")]
    )
    groupby_df[("Stim eye - Size Mm", "more than 50%")] = (
        groupby_df[("Stim eye - Size Mm", "count/size ratio")] >= 0.5
    )
    trial_stats_dict[str(participant_id)] = (
        groupby_df[
            [
                ("Block", ""),
                ("Trial no", ""),
                ("Trial type", ""),
                ("Stim eye - Size Mm", "count/size ratio"),
                ("Stim eye - Size Mm", "more than 50%"),
            ]
        ]
        .groupby([("Block", ""), ("Trial type", "")])
        .agg(
            {
                ("Stim eye - Size Mm", "count/size ratio"): [
                    "mean",
                    "std",
                    "min",
                    "max",
                ],
                ("Stim eye - Size Mm", "more than 50%"): "sum",
                ("Trial no", ""): "count",
            }
        )
        .reset_index()
    )
    
    #now for only stimulus phase
    
    data_df_new = data_df_new[data_df_new['Trial phase']=='stim']
    groupby_df = (
        data_df_new[
            ["Block", "Trial no", "Trial type", "Stim eye - Size Mm"]
        ]
        .groupby(["Block", "Trial no", "Trial type"])
        .agg(["count", "size"])
        .reset_index()
    )
    groupby_df[("Stim eye - Size Mm", "count/size ratio")] = (
        groupby_df[("Stim eye - Size Mm", "count")]
        / groupby_df[("Stim eye - Size Mm", "size")]
    )
    groupby_df[("Stim eye - Size Mm", "more than 50%")] = (
        groupby_df[("Stim eye - Size Mm", "count/size ratio")] >= 0.5
    )
    stim_stats_dict[str(participant_id)] = (
        groupby_df[
            [
                ("Block", ""),
                ("Trial no", ""),
                ("Trial type", ""),
                ("Stim eye - Size Mm", "count/size ratio"),
                ("Stim eye - Size Mm", "more than 50%"),
            ]
        ]
        .groupby([("Block", ""), ("Trial type", "")])
        .agg(
            {
                ("Stim eye - Size Mm", "count/size ratio"): [
                    "mean",
                    "std",
                    "min",
                    "max",
                ],
                ("Stim eye - Size Mm", "more than 50%"): "sum",
                ("Trial no", ""): "count",
            }
        )
        .reset_index()
    )
    data_df_new = []
    groupby_df = []

### Participant 200

In [121]:
print('Statistics for whole trials')
trial_stats_dict["200"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.585646  0.002311  0.580184  0.588175   
1      0        l-m           0.586668  0.001849  0.582227  0.588355   
2      0        lms           0.587131  0.001470  0.584270  0.588957   
3      0        mel           0.585869  0.002116  0.582227  0.588776   
4      0          s           0.586854  0.001285  0.583673  0.588355   
5      1       flux           0.585521  0.003147  0.577551  0.588355   
6      1        l-m           0.586344  0.001255  0.583673  0.587935   
7      1        lms           0.587046  0.001744  0.582653  0.589377   
8      1        mel           0.587052  0.001028  0.585466  0.588355   
9      1          s           0.585749  0.001506  0.582822  0.587755   
10     2       flux           0.584406  0.002474  0.580020  0.587334   
11     2        l-m           0.585587  0.003560  0.576531  0.588776   
12     2        lms           0.586152  0.001493  0.582653  0.587935   
13     2        mel           0.584176  0.004378  0.574055  0.588355   
14     2          s           0.584465  0.005407  0.572449  0.588355   
15     3       flux           0.587347  0.001184  0.584694  0.588776   
16     3        l-m           0.586957  0.001770  0.583673  0.588776   
17     3        lms           0.587812  0.001059  0.585890  0.589377   
18     3        mel           0.587341  0.001071  0.585291  0.589377   
19     3          s           0.586320  0.001703  0.582653  0.588355   
20     4       flux           0.586711  0.001498  0.583673  0.588355   
21     4        l-m           0.586013  0.002479  0.581800  0.588957   
22     4        lms           0.587267  0.001133  0.584694  0.588355   
23     4        mel           0.586788  0.001485  0.584270  0.588355   
24     4          s           0.585588  0.003371  0.576531  0.587755   
25     5       flux           0.584098  0.003628  0.577982  0.588175   
26     5        l-m           0.583036  0.007311  0.563265  0.588355   
27     5        lms           0.584444  0.003000  0.577982  0.587334   
28     5        mel           0.584668  0.003466  0.579162  0.588355   
29     5          s           0.582559  0.004281  0.574490  0.587334   
30     6       flux           0.586055  0.002560  0.582227  0.588776   
31     6        l-m           0.581972  0.005633  0.569969  0.588355   
32     6        lms           0.583877  0.005514  0.569827  0.587755   
33     6        mel           0.585894  0.002867  0.578571  0.588776   
34     6          s           0.573180  0.037137  0.468367  0.588776   
35     7       flux           0.587485  0.000912  0.585714  0.588776   
36     7        l-m           0.587016  0.000899  0.585291  0.588175   
37     7        lms           0.587595  0.000850  0.586106  0.588957   
38     7        mel           0.587112  0.001238  0.584270  0.588355   
39     7          s           0.586608  0.001460  0.583673  0.588355   
40     8       flux           0.585115  0.003759  0.573034  0.587334   
41     8        l-m           0.587014  0.001682  0.584270  0.589377   
42     8        lms           0.549785  0.141122  0.039712  0.588776   
43     8        mel           0.585845  0.002077  0.579592  0.588776   
44     8          s           0.585262  0.002546  0.581040  0.588776   
45     9       flux           0.581138  0.008564  0.565306  0.588355   
46     9        l-m           0.580912  0.007281  0.568648  0.588175   
47     9        lms           0.579961  0.011184  0.556345  0.589195   
48     9        mel           0.583590  0.005096  0.572895  0.587334   
49     9          s           0.581901  0.008149  0.567623  0.588776   
50    10       flux           0.586523  0.002330  0.581372  0.588776   
51    10        l-m           0.588116  0.000880  0.586137  0.589377   
52    10        lms           0.585408  0.0

### Participant 201

In [122]:
print('Statistics for whole trials')
trial_stats_dict["201"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.529266  0.057816  0.446374  0.587084   
1      0        l-m           0.525290  0.092868  0.360572  0.586735   
2      0        lms           0.440079  0.155431  0.235714  0.578571   
3      0        mel           0.424165  0.106904  0.297959  0.559184   
4      0          s           0.475252  0.101295  0.356122  0.586313   
5      1       flux           0.571568  0.014195  0.541544  0.586735   
6      1        l-m           0.573693  0.008224  0.559063  0.584270   
7      1        lms           0.569883  0.016094  0.542390  0.585291   
8      1        mel           0.569172  0.014086  0.532990  0.582653   
9      1          s           0.559262  0.026219  0.497980  0.585714   
10     2       flux           0.577925  0.011421  0.547959  0.587755   
11     2        l-m           0.581137  0.007607  0.563265  0.587755   
12     2        lms           0.576939  0.013440  0.551859  0.588355   
13     2        mel           0.575206  0.010386  0.549540  0.586313   
14     2          s           0.580280  0.006163  0.566769  0.587755   
15     3       flux           0.572176  0.009983  0.556122  0.586735   
16     3        l-m           0.569327  0.018854  0.544898  0.588776   
17     3        lms           0.570161  0.012743  0.553170  0.586313   
18     3        mel           0.575926  0.014085  0.552147  0.587755   
19     3          s           0.576261  0.015533  0.534694  0.587935   
20     4       flux           0.532701  0.068313  0.373469  0.587755   
21     4        l-m           0.563978  0.030020  0.505607  0.588355   
22     4        lms           0.521270  0.103613  0.272727  0.589377   
23     4        mel           0.550181  0.057881  0.394280  0.588595   
24     4          s           0.553432  0.042720  0.459735  0.587156   
25     5       flux           0.467174  0.119514  0.228805  0.580235   
26     5        l-m           0.398179  0.179454  0.147959  0.567347   
27     5        lms           0.432121  0.125380  0.235812  0.580612   
28     5        mel           0.456867  0.110442  0.267620  0.587755   
29     5          s           0.454714  0.147879  0.143003  0.582653   
30     6       flux           0.578116  0.006453  0.564862  0.584270   
31     6        l-m           0.569066  0.017079  0.537755  0.587334   
32     6        lms           0.573614  0.015749  0.540347  0.586735   
33     6        mel           0.576841  0.011047  0.553626  0.585714   
34     6          s           0.564993  0.014432  0.539326  0.581633   
35     7       flux           0.569659  0.022644  0.529111  0.588175   
36     7        l-m           0.563766  0.041869  0.452041  0.588355   
37     7        lms           0.546849  0.061322  0.411645  0.587334   
38     7        mel           0.549694  0.059329  0.419816  0.586735   
39     7          s           0.530733  0.091906  0.327551  0.588355   
40     8       flux           0.586029  0.001376  0.583673  0.588175   
41     8        l-m           0.582643  0.010529  0.553626  0.588776   
42     8        lms           0.583770  0.004482  0.575510  0.587755   
43     8        mel           0.584339  0.006907  0.565306  0.589796   
44     8          s           0.580383  0.011378  0.559755  0.589980   
45     9       flux           0.580836  0.011283  0.558163  0.588355   
46     9        l-m           0.586446  0.002785  0.579162  0.589377   
47     9        lms           0.585032  0.001850  0.582227  0.588355   
48     9        mel           0.583419  0.007382  0.568367  0.588355   
49     9          s           0.585129  0.005142  0.570991  0.588355   
50    10       flux           0.578082  0.009877  0.554536  0.587935   
51    10        l-m           0.579177  0.007628  0.563710  0.587935   
52    10        lms           0.573290  0.0

### Participant 202

In [123]:
print('Statistics for whole trials')
trial_stats_dict["202"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.574193  0.021486  0.526531  0.588776   
1      0        l-m           0.584285  0.003635  0.577120  0.587755   
2      0        lms           0.581447  0.013519  0.546476  0.588355   
3      0        mel           0.577857  0.017654  0.532653  0.588355   
4      0          s           0.582055  0.005539  0.571575  0.588776   
5      1       flux           0.569850  0.030193  0.500511  0.588776   
6      1        l-m           0.561136  0.042179  0.462245  0.588175   
7      1        lms           0.572246  0.040446  0.459735  0.589377   
8      1        mel           0.564525  0.072518  0.358163  0.589377   
9      1          s           0.567365  0.049921  0.426531  0.588776   
10     2       flux           0.552577  0.060767  0.388776  0.588776   
11     2        l-m           0.539263  0.084725  0.360572  0.588355   
12     2        lms           0.515549  0.098157  0.291539  0.588355   
13     2        mel           0.565951  0.037965  0.462717  0.588776   
14     2          s           0.544249  0.089406  0.315628  0.587755   
15     3       flux           0.577658  0.010392  0.557594  0.588063   
16     3        l-m           0.584626  0.006898  0.565884  0.589377   
17     3        lms           0.578614  0.009999  0.560776  0.586735   
18     3        mel           0.582543  0.006431  0.570991  0.588355   
19     3          s           0.584133  0.003671  0.576531  0.587935   
20     4       flux           0.576062  0.012982  0.552605  0.588355   
21     4        l-m           0.584082  0.002649  0.579592  0.587334   
22     4        lms           0.573514  0.021623  0.519388  0.587334   
23     4        mel           0.575204  0.021726  0.520940  0.587576   
24     4          s           0.572510  0.021111  0.530071  0.587755   
25     5       flux           0.544689  0.060590  0.440816  0.588355   
26     5        l-m           0.543318  0.066395  0.396534  0.588175   
27     5        lms           0.557543  0.036629  0.498980  0.587755   
28     5        mel           0.561920  0.031919  0.493878  0.587755   
29     5          s           0.551177  0.041245  0.464760  0.587935   
30     6       flux           0.558040  0.053490  0.421859  0.586735   
31     6        l-m           0.541220  0.085195  0.338776  0.588355   
32     6        lms           0.567341  0.034731  0.484200  0.587755   
33     6        mel           0.581386  0.007950  0.567347  0.588355   
34     6          s           0.561136  0.032595  0.505618  0.587334   
35     7       flux           0.584345  0.008886  0.559633  0.588776   
36     7        l-m           0.568547  0.034601  0.498980  0.589377   
37     7        lms           0.586162  0.002985  0.580235  0.589377   
38     7        mel           0.586710  0.001911  0.582227  0.588776   
39     7          s           0.586319  0.002835  0.580184  0.589377   
40     8       flux           0.578249  0.011290  0.553061  0.588776   
41     8        l-m           0.583218  0.010745  0.554082  0.588957   
42     8        lms           0.575308  0.017314  0.536735  0.587755   
43     8        mel           0.579259  0.016443  0.538776  0.588776   
44     8          s           0.579248  0.013360  0.545918  0.587935   
45     9       flux           0.586912  0.001110  0.584694  0.588355   
46     9        l-m           0.587034  0.001652  0.583248  0.588957   
47     9        lms           0.587977  0.001068  0.585533  0.588957   
48     9        mel           0.587298  0.001418  0.585291  0.589377   
49     9          s           0.584694  0.008285  0.561224  0.588355   
50    10       flux           0.451621  0.187708  0.043922  0.587755   
51    10        l-m           0.409820  0.198522  0.041837  0.587935   
52    10        lms           0.450959  0.1

### Participant 204

In [124]:
print('Statistics for whole trials')
trial_stats_dict["204"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.334153  0.124052  0.159346  0.515400   
1      0        l-m           0.345828  0.120167  0.119388  0.516327   
2      0        lms           0.390501  0.086170  0.253320  0.536188   
3      0        mel           0.349284  0.098104  0.215526  0.503575   
4      0          s           0.344764  0.153032  0.164286  0.559184   
5      1       flux           0.501411  0.066504  0.365949  0.577120   
6      1        l-m           0.476424  0.094342  0.254341  0.567485   
7      1        lms           0.448270  0.137471  0.167347  0.578571   
8      1        mel           0.439193  0.148800  0.208970  0.584867   
9      1          s           0.476491  0.113193  0.250765  0.580020   
10     2       flux           0.485781  0.119196  0.210419  0.578141   
11     2        l-m           0.539466  0.030707  0.495403  0.584694   
12     2        lms           0.499857  0.122305  0.265036  0.583248   
13     2        mel           0.452762  0.110440  0.291837  0.577551   
14     2          s           0.518238  0.067305  0.347959  0.578141   
15     3       flux           0.474850  0.100071  0.288776  0.567347   
16     3        l-m           0.487144  0.043537  0.398573  0.553061   
17     3        lms           0.471555  0.058511  0.348980  0.539796   
18     3        mel           0.506538  0.044396  0.399387  0.572012   
19     3          s           0.489948  0.046085  0.397344  0.548980   
20     4       flux           0.445035  0.086732  0.322778  0.531154   
21     4        l-m           0.408523  0.114132  0.241062  0.548980   
22     4        lms           0.434611  0.147251  0.072449  0.543412   
23     4        mel           0.430944  0.128431  0.146939  0.562245   
24     4          s           0.471856  0.052037  0.375894  0.553517   
25     5       flux           0.404628  0.080124  0.295918  0.560204   
26     5        l-m           0.411654  0.110727  0.167347  0.525510   
27     5        lms           0.354583  0.124699  0.130879  0.505618   
28     5        mel           0.425168  0.041321  0.374872  0.486735   
29     5          s           0.425351  0.107868  0.156282  0.516327   
30     6       flux           0.385049  0.133889  0.185728  0.546939   
31     6        l-m           0.414482  0.101252  0.277835  0.573469   
32     6        lms           0.334238  0.137656  0.070336  0.467824   
33     6        mel           0.436716  0.125622  0.167347  0.566905   
34     6          s           0.363189  0.125731  0.096840  0.511224   
35     7       flux           0.302963  0.109837  0.139869  0.537634   
36     7        l-m           0.277475  0.128066  0.038462  0.487232   
37     7        lms           0.311452  0.161945  0.019608  0.556237   
38     7        mel           0.306217  0.147988  0.064987  0.530612   
39     7          s           0.298452  0.143022  0.093872  0.517875   
40     8       flux           0.359597  0.129887  0.177370  0.556428   
41     8        l-m           0.413087  0.084745  0.276531  0.533129   
42     8        lms           0.414102  0.096274  0.242610  0.560327   
43     8        mel           0.400225  0.071945  0.275510  0.496425   
44     8          s           0.458168  0.092232  0.283963  0.564155   
45     9       flux           0.399055  0.142260  0.068437  0.557297   
46     9        l-m           0.381313  0.127247  0.153218  0.526047   
47     9        lms           0.369903  0.149253  0.039796  0.505102   
48     9        mel           0.400559  0.091583  0.213048  0.510725   
49     9          s           0.363028  0.159576  0.036810  0.546939   
50    10       flux           0.537815  0.023734  0.481595  0.568367   
51    10        l-m           0.530374  0.014588  0.506122  0.554082   
52    10        lms           0.532964  0.0

### Participant 205

In [125]:
print('Statistics for whole trials')
trial_stats_dict["205"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.291130  0.093866  0.147959  0.395301   
1      0        l-m           0.238177  0.147997  0.046987  0.473469   
2      0        lms           0.231310  0.137588  0.013265  0.436290   
3      0        mel           0.137583  0.114040  0.000000  0.315306   
4      0          s           0.180028  0.118951  0.051020  0.443878   
5      1       flux           0.327086  0.148841  0.164969  0.528571   
6      1        l-m           0.305395  0.132800  0.095821  0.501020   
7      1        lms           0.295512  0.212422  0.000000  0.537755   
8      1        mel           0.270529  0.144853  0.000000  0.439796   
9      1          s           0.312962  0.143774  0.098980  0.548980   
10     2       flux           0.283808  0.207518  0.072375  0.552605   
11     2        l-m           0.270267  0.158593  0.000000  0.482125   
12     2        lms           0.249472  0.184588  0.025536  0.542390   
13     2        mel           0.209835  0.142396  0.058163  0.508163   
14     2          s           0.244123  0.219162  0.010215  0.552605   
15     3       flux           0.285509  0.104093  0.157303  0.502935   
16     3        l-m           0.252292  0.133902  0.101020  0.531154   
17     3        lms           0.334737  0.106454  0.211224  0.518367   
18     3        mel           0.277065  0.159748  0.065306  0.539796   
19     3          s           0.321808  0.123569  0.112245  0.525026   
20     4       flux           0.135752  0.073386  0.036772  0.267620   
21     4        l-m           0.167665  0.158229  0.000000  0.428571   
22     4        lms           0.111507  0.103385  0.000000  0.347959   
23     4        mel           0.160036  0.087889  0.036772  0.335714   
24     4          s           0.136306  0.097193  0.000000  0.272727   
25     5       flux           0.193295  0.150458  0.000000  0.402041   
26     5        l-m           0.229457  0.145345  0.000000  0.435714   
27     5        lms           0.247862  0.133408  0.057201  0.403061   
28     5        mel           0.268040  0.143661  0.082822  0.503061   
29     5          s           0.282922  0.122132  0.109295  0.491318   
30     6       flux           0.347103  0.143281  0.114402  0.512720   
31     6        l-m           0.365983  0.149067  0.141982  0.540816   
32     6        lms           0.370282  0.134908  0.134694  0.552605   
33     6        mel           0.359243  0.131777  0.096016  0.502554   
34     6          s           0.353540  0.123966  0.139653  0.555669   
35     7       flux           0.434127  0.074010  0.348315  0.511747   
36     7        l-m           0.457844  0.034364  0.425510  0.509704   
37     7        lms           0.450985  0.089124  0.352761  0.562622   
38     7        mel           0.331248  0.120468  0.175510  0.469867   
39     7          s           0.387884  0.064706  0.298264  0.474490   
40     8       flux           0.469047  0.053210  0.366972  0.545455   
41     8        l-m           0.405038  0.078608  0.273469  0.511224   
42     8        lms           0.441343  0.093154  0.261224  0.548924   
43     8        mel           0.352323  0.112970  0.201020  0.542857   
44     8          s           0.391783  0.108172  0.229826  0.546381   
45     9       flux           0.299207  0.146439  0.000000  0.482618   
46     9        l-m           0.310070  0.108599  0.150000  0.463812   
47     9        lms           0.355152  0.139034  0.082653  0.509184   
48     9        mel           0.387092  0.088250  0.260204  0.553517   
49     9          s           0.294549  0.122008  0.108053  0.436735   
50    10       flux           0.318117  0.186441  0.043922  0.549080   
51    10        l-m           0.259875  0.161185  0.041837  0.548980   
52    10        lms           0.329612  0.2

### Participant 206

In [126]:
print('Statistics for whole trials')
trial_stats_dict["206"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.543953  0.048742  0.412844  0.580612   
1      0        l-m           0.552131  0.032354  0.489275  0.584694   
2      0        lms           0.558167  0.015058  0.534219  0.578141   
3      0        mel           0.544835  0.023628  0.494898  0.569388   
4      0          s           0.544411  0.053520  0.418367  0.580184   
5      1       flux           0.382719  0.077378  0.335509  0.586735   
6      1        l-m           0.361508  0.080733  0.311360  0.586735   
7      1        lms           0.392411  0.102642  0.314694  0.588776   
8      1        mel           0.397347  0.129491  0.250965  0.581633   
9      1          s           0.392667  0.098502  0.333333  0.580184   
10     2       flux           0.568379  0.010524  0.551020  0.582031   
11     2        l-m           0.569533  0.009835  0.558733  0.588355   
12     2        lms           0.567148  0.026234  0.496425  0.587334   
13     2        mel           0.546765  0.086087  0.302349  0.581800   
14     2          s           0.545615  0.060267  0.394898  0.586137   
15     3       flux           0.535859  0.043811  0.477495  0.585117   
16     3        l-m           0.511182  0.090184  0.303061  0.586313   
17     3        lms           0.493921  0.122035  0.199796  0.583673   
18     3        mel           0.519362  0.061325  0.397959  0.581040   
19     3          s           0.496573  0.104195  0.255624  0.580612   
20     4       flux           0.546239  0.054100  0.404494  0.579824   
21     4        l-m           0.565551  0.017917  0.533197  0.582653   
22     4        lms           0.564172  0.013449  0.542857  0.583673   
23     4        mel           0.562805  0.023165  0.522936  0.586735   
24     4          s           0.568119  0.018509  0.535714  0.587755   
25     5       flux           0.468808  0.101297  0.338446  0.577551   
26     5        l-m           0.482559  0.076128  0.341837  0.564286   
27     5        lms           0.512124  0.048679  0.438776  0.559755   
28     5        mel           0.520522  0.044223  0.427988  0.581205   
29     5          s           0.512134  0.055127  0.417773  0.569969   
30     6       flux           0.500497  0.092215  0.274210  0.580020   
31     6        l-m           0.499017  0.057083  0.367722  0.549080   
32     6        lms           0.494944  0.072337  0.321063  0.554648   
33     6        mel           0.452288  0.077704  0.351682  0.567347   
34     6          s           0.495817  0.056555  0.400409  0.558733   
35     7       flux           0.464128  0.146484  0.183953  0.587755   
36     7        l-m           0.511646  0.122419  0.186925  0.586313   
37     7        lms           0.442510  0.188358  0.109295  0.585714   
38     7        mel           0.404088  0.184429  0.106122  0.580184   
39     7          s           0.449960  0.161883  0.213265  0.585714   
40     8       flux           0.570681  0.010691  0.549439  0.585117   
41     8        l-m           0.563397  0.016263  0.533673  0.587755   
42     8        lms           0.560404  0.020349  0.521429  0.580612   
43     8        mel           0.575410  0.008070  0.557143  0.584270   
44     8          s           0.563255  0.036179  0.464760  0.588776   
45     9       flux           0.576686  0.019524  0.532176  0.588776   
46     9        l-m           0.582949  0.006289  0.572597  0.587935   
47     9        lms           0.585345  0.003975  0.575510  0.588776   
48     9        mel           0.582731  0.008041  0.567347  0.589377   
49     9          s           0.580148  0.019057  0.526047  0.588175   
50    10       flux           0.555627  0.039474  0.477017  0.587755   
51    10        l-m           0.540297  0.056699  0.439224  0.588776   
52    10        lms           0.568359  0.0

### Participant 207

In [127]:
print('Statistics for whole trials')
trial_stats_dict["207"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.482268  0.110787  0.228571  0.566327   
1      0        l-m           0.536288  0.043526  0.458163  0.573469   
2      0        lms           0.493327  0.122175  0.176351  0.573469   
3      0        mel           0.491627  0.152364  0.083845  0.576098   
4      0          s           0.525793  0.076233  0.325510  0.574055   
5      1       flux           0.538043  0.012828  0.513790  0.552497   
6      1        l-m           0.545736  0.015948  0.524490  0.570408   
7      1        lms           0.537735  0.021399  0.494898  0.565749   
8      1        mel           0.526300  0.057594  0.372069  0.560652   
9      1          s           0.544495  0.013387  0.515832  0.560204   
10     2       flux           0.327135  0.156248  0.000000  0.531633   
11     2        l-m           0.447289  0.077585  0.321757  0.530133   
12     2        lms           0.412993  0.134618  0.086823  0.518897   
13     2        mel           0.421160  0.154186  0.000000  0.555669   
14     2          s           0.386534  0.123265  0.132653  0.518367   
15     3       flux           0.382915  0.126622  0.170421  0.540816   
16     3        l-m           0.358640  0.089770  0.212029  0.495413   
17     3        lms           0.402705  0.075445  0.295501  0.535168   
18     3        mel           0.379541  0.114805  0.199183  0.541837   
19     3          s           0.415341  0.086889  0.255363  0.514286   
20     4       flux           0.128773  0.100746  0.000000  0.349336   
21     4        l-m           0.190095  0.094164  0.047959  0.365953   
22     4        lms           0.214201  0.136551  0.008163  0.416327   
23     4        mel           0.176214  0.106287  0.000000  0.365953   
24     4          s           0.141189  0.131074  0.012232  0.416327   
25     5       flux           0.294787  0.104638  0.130746  0.428571   
26     5        l-m           0.305361  0.116662  0.105209  0.447395   
27     5        lms           0.252630  0.149790  0.000000  0.412245   
28     5        mel           0.308491  0.120226  0.114286  0.492857   
29     5          s           0.245966  0.137823  0.037717  0.483673   
30     6       flux           0.422008  0.140532  0.086735  0.547959   
31     6        l-m           0.426332  0.120993  0.181448  0.551020   
32     6        lms           0.375588  0.142907  0.165475  0.533197   
33     6        mel           0.381379  0.113447  0.129724  0.507143   
34     6          s           0.443501  0.105925  0.186735  0.556122   
35     7       flux           0.398782  0.139886  0.052147  0.561224   
36     7        l-m           0.447988  0.079465  0.336057  0.564417   
37     7        lms           0.419934  0.104506  0.231898  0.557712   
38     7        mel           0.399321  0.158749  0.002843  0.561798   
39     7          s           0.386007  0.182251  0.070408  0.559184   
40     8       flux           0.407837  0.139579  0.113381  0.539326   
41     8        l-m           0.372681  0.194395  0.085714  0.551583   
42     8        lms           0.422041  0.155958  0.032653  0.547959   
43     8        mel           0.379630  0.148610  0.088866  0.551583   
44     8          s           0.385334  0.176648  0.057143  0.534694   
45     9       flux           0.413376  0.176537  0.039755  0.555102   
46     9        l-m           0.511581  0.013106  0.498468  0.533673   
47     9        lms           0.411683  0.140331  0.184883  0.541837   
48     9        mel           0.471829  0.056839  0.355465  0.536261   
49     9          s           0.378691  0.198453  0.035824  0.556691   
50    10       flux           0.356741  0.097726  0.166496  0.477017   
51    10        l-m           0.390122  0.099545  0.235714  0.513292   
52    10        lms           0.286845  0.1

### Participant 209

In [128]:
print('Statistics for whole trials')
trial_stats_dict["209"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.221087  0.103753  0.009184  0.376915   
1      0        l-m           0.201046  0.147249  0.000000  0.458163   
2      0        lms           0.181878  0.143888  0.000000  0.383044   
3      0        mel           0.236619  0.147836  0.017365  0.469388   
4      0          s           0.195413  0.153929  0.000000  0.418534   
5      1       flux           0.238458  0.160376  0.000000  0.576531   
6      1        l-m           0.223056  0.164333  0.042857  0.486735   
7      1        lms           0.260162  0.139297  0.000000  0.530071   
8      1        mel           0.288901  0.174226  0.000000  0.588355   
9      1          s           0.211563  0.169855  0.000000  0.587334   
10     2       flux           0.343203  0.144818  0.090816  0.566536   
11     2        l-m           0.379838  0.162783  0.181818  0.576098   
12     2        lms           0.312289  0.172741  0.044898  0.557712   
13     2        mel           0.347851  0.194889  0.040816  0.564286   
14     2          s           0.317398  0.164323  0.026558  0.541369   
15     3       flux           0.204926  0.177041  0.012270  0.451943   
16     3        l-m           0.129271  0.122363  0.000000  0.400000   
17     3        lms           0.192075  0.178857  0.000000  0.440816   
18     3        mel           0.224586  0.201990  0.000000  0.548057   
19     3          s           0.175141  0.135311  0.029592  0.461224   
20     4       flux           0.268422  0.153478  0.000000  0.517347   
21     4        l-m           0.238856  0.119246  0.033673  0.420570   
22     4        lms           0.285490  0.197142  0.015306  0.580184   
23     4        mel           0.269864  0.174859  0.000000  0.534694   
24     4          s           0.290824  0.179859  0.000000  0.523469   
25     5       flux           0.505234  0.055056  0.387755  0.552838   
26     5        l-m           0.522656  0.028254  0.456033  0.553061   
27     5        lms           0.518806  0.039222  0.441267  0.567347   
28     5        mel           0.513835  0.039748  0.424490  0.552605   
29     5          s           0.486471  0.052168  0.386108  0.533197   
30     6       flux           0.071367  0.098485  0.000000  0.281920   
31     6        l-m           0.037669  0.059717  0.000000  0.195918   
32     6        lms           0.043616  0.062657  0.000000  0.167689   
33     6        mel           0.105877  0.157335  0.000000  0.409786   
34     6          s           0.063750  0.097885  0.000000  0.250765   
35     7       flux           0.033302  0.083719  0.000000  0.267620   
36     7        l-m           0.009795  0.018774  0.000000  0.048980   
37     7        lms           0.025102  0.074496  0.000000  0.236735   
38     7        mel           0.007037  0.011822  0.000000  0.031633   
39     7          s           0.047245  0.063642  0.000000  0.182840   
40     8       flux           0.358791  0.178025  0.165306  0.585714   
41     8        l-m           0.431452  0.125310  0.216327  0.572012   
42     8        lms           0.411628  0.137648  0.190622  0.573034   
43     8        mel           0.417239  0.094582  0.251020  0.581205   
44     8          s           0.396091  0.181762  0.080695  0.567926   
45     9       flux           0.323920  0.122727  0.116327  0.475996   
46     9        l-m           0.308668  0.128453  0.080612  0.476483   
47     9        lms           0.396858  0.100693  0.258317  0.542304   
48     9        mel           0.369160  0.164339  0.009193  0.559184   
49     9          s           0.415941  0.065541  0.318043  0.538776   
50    10       flux           0.388060  0.142564  0.161224  0.534694   
51    10        l-m           0.381885  0.114727  0.195097  0.487755   
52    10        lms           0.341639  0.1

### Participant 210

In [129]:
print('Statistics for whole trials')
trial_stats_dict["210"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.493934  0.044953  0.408163  0.555669   
1      0        l-m           0.473656  0.074726  0.342536  0.553626   
2      0        lms           0.494385  0.067865  0.372829  0.580645   
3      0        mel           0.476561  0.063839  0.368744  0.558733   
4      0          s           0.499998  0.044950  0.429592  0.576531   
5      1       flux           0.324718  0.138128  0.080695  0.481409   
6      1        l-m           0.304134  0.154312  0.020387  0.569827   
7      1        lms           0.343453  0.112741  0.158325  0.547959   
8      1        mel           0.310196  0.103284  0.158163  0.459184   
9      1          s           0.387319  0.073490  0.275229  0.517347   
10     2       flux           0.458422  0.077974  0.284694  0.547959   
11     2        l-m           0.464179  0.089907  0.292857  0.551583   
12     2        lms           0.477832  0.059154  0.356513  0.541284   
13     2        mel           0.438267  0.090487  0.295918  0.527068   
14     2          s           0.467094  0.061100  0.365953  0.547959   
15     3       flux           0.344725  0.174789  0.104082  0.547959   
16     3        l-m           0.326238  0.148077  0.127681  0.550459   
17     3        lms           0.299750  0.200475  0.030333  0.562245   
18     3        mel           0.326624  0.194589  0.047910  0.543878   
19     3          s           0.317217  0.154057  0.087755  0.510204   
20     4       flux           0.288261  0.187331  0.051935  0.563841   
21     4        l-m           0.363817  0.113817  0.146939  0.479103   
22     4        lms           0.260366  0.111909  0.126660  0.478571   
23     4        mel           0.348633  0.121368  0.138776  0.478039   
24     4          s           0.257965  0.114720  0.127291  0.483673   
25     5       flux           0.296780  0.147287  0.138776  0.551583   
26     5        l-m           0.409907  0.143503  0.192865  0.582227   
27     5        lms           0.251364  0.170745  0.026558  0.525026   
28     5        mel           0.288860  0.142735  0.022449  0.487755   
29     5          s           0.346461  0.158144  0.030892  0.565306   
30     6       flux           0.312099  0.155037  0.101020  0.522449   
31     6        l-m           0.396508  0.118201  0.159346  0.532176   
32     6        lms           0.393873  0.156956  0.107143  0.526047   
33     6        mel           0.308711  0.129207  0.138776  0.498980   
34     6          s           0.351877  0.140806  0.131498  0.512742   
35     7       flux           0.227892  0.091893  0.113265  0.383187   
36     7        l-m           0.277054  0.082665  0.133810  0.407747   
37     7        lms           0.326453  0.127969  0.103166  0.496939   
38     7        mel           0.283622  0.153928  0.090909  0.540816   
39     7          s           0.256323  0.123834  0.101020  0.480612   
40     8       flux           0.291753  0.099484  0.174668  0.468846   
41     8        l-m           0.286146  0.113684  0.128571  0.504082   
42     8        lms           0.268318  0.141665  0.000000  0.450000   
43     8        mel           0.229198  0.116165  0.042857  0.380224   
44     8          s           0.244989  0.139509  0.128571  0.575510   
45     9       flux           0.296191  0.150356  0.093878  0.489796   
46     9        l-m           0.437959  0.067245  0.313585  0.560776   
47     9        lms           0.284772  0.122725  0.141837  0.443878   
48     9        mel           0.318047  0.130939  0.164286  0.536188   
49     9          s           0.366451  0.082913  0.220408  0.521472   
50    10       flux           0.239110  0.173101  0.017329  0.525994   
51    10        l-m           0.336730  0.122704  0.161060  0.513265   
52    10        lms           0.179684  0.1

### Participant 211

In [130]:
print('Statistics for whole trials')
trial_stats_dict["211"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.540129  0.029168  0.497446  0.588776   
1      0        l-m           0.561848  0.014619  0.541369  0.581205   
2      0        lms           0.557071  0.026719  0.494129  0.580612   
3      0        mel           0.534910  0.048167  0.420838  0.586735   
4      0          s           0.518464  0.035170  0.459653  0.555102   
5      1       flux           0.410159  0.179134  0.176711  0.584555   
6      1        l-m           0.482379  0.110489  0.250000  0.575077   
7      1        lms           0.459160  0.121172  0.257900  0.578571   
8      1        mel           0.451789  0.156970  0.205912  0.587513   
9      1          s           0.495211  0.092609  0.307143  0.581633   
10     2       flux           0.549606  0.034468  0.489796  0.581205   
11     2        l-m           0.565564  0.012215  0.543412  0.577120   
12     2        lms           0.545629  0.026065  0.497446  0.581040   
13     2        mel           0.562716  0.020018  0.523469  0.588175   
14     2          s           0.544896  0.029487  0.489275  0.581040   
15     3       flux           0.564862  0.028498  0.491892  0.588776   
16     3        l-m           0.565562  0.016223  0.536735  0.585291   
17     3        lms           0.549770  0.027952  0.499490  0.578571   
18     3        mel           0.559249  0.017238  0.533673  0.577551   
19     3          s           0.560349  0.030278  0.492355  0.583673   
20     4       flux           0.522450  0.059051  0.375894  0.570552   
21     4        l-m           0.474448  0.177112  0.071356  0.576531   
22     4        lms           0.545273  0.034288  0.470408  0.582059   
23     4        mel           0.465025  0.161513  0.085714  0.580184   
24     4          s           0.521787  0.088095  0.285714  0.583845   
25     5       flux           0.506725  0.055054  0.427115  0.574055   
26     5        l-m           0.486905  0.065142  0.357873  0.566327   
27     5        lms           0.528358  0.048669  0.467824  0.581372   
28     5        mel           0.499960  0.070053  0.364286  0.573034   
29     5          s           0.492519  0.075744  0.358818  0.574490   
30     6       flux           0.467302  0.063560  0.341837  0.539796   
31     6        l-m           0.424343  0.110090  0.308163  0.563841   
32     6        lms           0.480244  0.081884  0.300306  0.572012   
33     6        mel           0.448600  0.085320  0.355759  0.584694   
34     6          s           0.502016  0.076688  0.360204  0.577551   
35     7       flux           0.491861  0.094434  0.278571  0.582227   
36     7        l-m           0.460084  0.133143  0.238776  0.585117   
37     7        lms           0.504395  0.050462  0.436605  0.579755   
38     7        mel           0.530019  0.032037  0.490816  0.581205   
39     7          s           0.474506  0.099790  0.293878  0.586313   
40     8       flux           0.554137  0.025050  0.501529  0.574619   
41     8        l-m           0.556013  0.046465  0.434694  0.586313   
42     8        lms           0.563041  0.024224  0.507645  0.583248   
43     8        mel           0.550007  0.038683  0.467890  0.583078   
44     8          s           0.554018  0.026496  0.494393  0.578571   
45     9       flux           0.581131  0.013775  0.543878  0.588355   
46     9        l-m           0.575687  0.016114  0.540265  0.587755   
47     9        lms           0.578606  0.009191  0.566769  0.587755   
48     9        mel           0.574336  0.020957  0.528090  0.589377   
49     9          s           0.579846  0.007262  0.567347  0.588355   
50    10       flux           0.567585  0.019394  0.519918  0.587935   
51    10        l-m           0.577114  0.007071  0.569388  0.588355   
52    10        lms           0.571609  0.0

### Participant 212

In [131]:
print('Statistics for whole trials')
trial_stats_dict["212"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.465476  0.170248  0.139796  0.588776   
1      0        l-m           0.497295  0.135635  0.226531  0.588776   
2      0        lms           0.432850  0.145903  0.152905  0.585714   
3      0        mel           0.423180  0.166166  0.098059  0.587755   
4      0          s           0.456820  0.161150  0.144898  0.587755   
5      1       flux           0.401213  0.162715  0.095918  0.588776   
6      1        l-m           0.376368  0.192010  0.035714  0.588175   
7      1        lms           0.397812  0.202934  0.043922  0.588776   
8      1        mel           0.372205  0.207443  0.033639  0.586735   
9      1          s           0.370447  0.245114  0.000000  0.587755   
10     2       flux           0.498676  0.185085  0.000000  0.587755   
11     2        l-m           0.558603  0.066336  0.374490  0.588776   
12     2        lms           0.514410  0.175085  0.020492  0.588355   
13     2        mel           0.516627  0.182287  0.000000  0.589195   
14     2          s           0.422874  0.241276  0.000000  0.588355   
15     3       flux           0.587087  0.001708  0.583248  0.588776   
16     3        l-m           0.573812  0.023746  0.520408  0.588776   
17     3        lms           0.575288  0.021320  0.533129  0.588355   
18     3        mel           0.586405  0.002437  0.580020  0.588776   
19     3          s           0.585102  0.003022  0.580777  0.588355   
20     4       flux           0.561550  0.045733  0.467347  0.588063   
21     4        l-m           0.583239  0.006517  0.568367  0.589377   
22     4        lms           0.585863  0.003113  0.579162  0.588776   
23     4        mel           0.583912  0.004079  0.576531  0.588175   
24     4          s           0.578895  0.020357  0.521429  0.587755   
25     5       flux           0.579103  0.011026  0.555669  0.588776   
26     5        l-m           0.578484  0.007196  0.564286  0.587156   
27     5        lms           0.582494  0.005939  0.569530  0.588355   
28     5        mel           0.583870  0.005479  0.572449  0.589377   
29     5          s           0.569964  0.019270  0.529111  0.588776   
30     6       flux           0.435390  0.131733  0.140059  0.588355   
31     6        l-m           0.334064  0.183277  0.101020  0.550562   
32     6        lms           0.463220  0.104554  0.292135  0.587334   
33     6        mel           0.446629  0.132610  0.198980  0.586313   
34     6          s           0.491430  0.091248  0.310521  0.587334   
35     7       flux           0.551898  0.102126  0.261491  0.588776   
36     7        l-m           0.529414  0.085975  0.364658  0.588776   
37     7        lms           0.584561  0.002606  0.579592  0.586735   
38     7        mel           0.548356  0.067851  0.392638  0.588355   
39     7          s           0.578373  0.020189  0.522449  0.588776   
40     8       flux           0.582010  0.011290  0.555669  0.588776   
41     8        l-m           0.554312  0.055938  0.414286  0.587755   
42     8        lms           0.584927  0.003494  0.579162  0.588355   
43     8        mel           0.583692  0.007435  0.563710  0.588776   
44     8          s           0.580683  0.020344  0.522983  0.588355   
45     9       flux           0.560789  0.047773  0.431633  0.586313   
46     9        l-m           0.492256  0.137118  0.257143  0.586137   
47     9        lms           0.555641  0.063528  0.389173  0.586912   
48     9        mel           0.561519  0.052784  0.414286  0.588355   
49     9          s           0.572431  0.020718  0.518367  0.585714   
50    10       flux           0.560693  0.042924  0.450000  0.587084   
51    10        l-m           0.407540  0.240484  0.018443  0.588957   
52    10        lms           0.533940  0.1

### Participant 213

In [132]:
print('Statistics for whole trials')
trial_stats_dict["213"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.394958  0.164286  0.000000  0.586735   
1      0        l-m           0.427859  0.216280  0.000000  0.581205   
2      0        lms           0.294797  0.228149  0.000000  0.544343   
3      0        mel           0.308534  0.267620  0.000000  0.553626   
4      0          s           0.358891  0.207142  0.039837  0.587755   
5      1       flux           0.573166  0.014302  0.548519  0.588776   
6      1        l-m           0.521234  0.066680  0.375510  0.584270   
7      1        lms           0.556398  0.039936  0.457696  0.587755   
8      1        mel           0.564781  0.021021  0.523955  0.588355   
9      1          s           0.546557  0.086409  0.307143  0.588355   
10     2       flux           0.538973  0.088796  0.358529  0.589377   
11     2        l-m           0.549598  0.048626  0.444898  0.587935   
12     2        lms           0.547768  0.102487  0.259184  0.588776   
13     2        mel           0.479082  0.181737  0.113265  0.587755   
14     2          s           0.531785  0.104797  0.263534  0.588776   
15     3       flux           0.540781  0.082402  0.315306  0.589041   
16     3        l-m           0.477393  0.074629  0.352400  0.581633   
17     3        lms           0.512539  0.072811  0.378571  0.587755   
18     3        mel           0.531105  0.099065  0.267347  0.588355   
19     3          s           0.490430  0.129051  0.197757  0.588175   
20     4       flux           0.538781  0.077039  0.338776  0.584694   
21     4        l-m           0.531208  0.091173  0.297242  0.584694   
22     4        lms           0.520459  0.090589  0.323829  0.588776   
23     4        mel           0.542352  0.050950  0.429009  0.583673   
24     4          s           0.557256  0.045823  0.447395  0.587334   
25     5       flux           0.422249  0.137966  0.166496  0.587755   
26     5        l-m           0.415266  0.154994  0.169388  0.580448   
27     5        lms           0.427469  0.123960  0.231397  0.586735   
28     5        mel           0.407074  0.170722  0.036735  0.573469   
29     5          s           0.374206  0.206164  0.040858  0.566769   
30     6       flux           0.446901  0.147681  0.114286  0.576098   
31     6        l-m           0.306540  0.172716  0.045918  0.584694   
32     6        lms           0.398239  0.138793  0.185714  0.542304   
33     6        mel           0.361063  0.172855  0.085802  0.587334   
34     6          s           0.294056  0.201102  0.020429  0.588776   
35     7       flux           0.398135  0.159491  0.074414  0.589377   
36     7        l-m           0.307589  0.172449  0.059184  0.529592   
37     7        lms           0.328413  0.192450  0.029592  0.567347   
38     7        mel           0.249679  0.157108  0.072523  0.523469   
39     7          s           0.418507  0.091102  0.290816  0.546939   
40     8       flux           0.305448  0.238457  0.014300  0.588776   
41     8        l-m           0.240460  0.201419  0.000000  0.574490   
42     8        lms           0.281936  0.229727  0.009184  0.586735   
43     8        mel           0.250941  0.212449  0.012257  0.587334   
44     8          s           0.322946  0.246647  0.016327  0.588776   
45     9       flux           0.377466  0.153542  0.122574  0.574490   
46     9        l-m           0.387500  0.194315  0.110429  0.587334   
47     9        lms           0.494577  0.122664  0.166154  0.588776   
48     9        mel           0.487151  0.141436  0.193054  0.586313   
49     9          s           0.472512  0.127226  0.224719  0.587334   
50    10       flux           0.511298  0.097307  0.291837  0.588063   
51    10        l-m           0.456755  0.075196  0.337079  0.586735   
52    10        lms           0.541410  0.1

## Analysis of data resampled to 25 Hz after removing rows where the other eye was being recorded

Motivation: eyetracker was not recording at a frequency high enough to produce a 50 Hz signal. Let's check if situation improves when resampling to 25 Hz. The stimulation had frequency of 5 Hz and latency of pupil responses is about 100 ms, so perhaps 50 Hz frequency is not needed for the goal of this study.

In [133]:
data_dir = "./results/new/removed_resampled/"
trial_stats_dict = {}
stim_stats_dict = {}
for participant_id in participant_list:
    data_path = os.path.join(data_dir, str(participant_id) + "_25_removed_resampled_data.csv")
    data_df_new = pd.read_csv(data_path)

    groupby_df = (
        data_df_new[
            ["Block", "Trial no", "Trial type", "Trial phase", "Stim eye - Size Mm"]
        ]
        .groupby(["Block", "Trial no", "Trial type"])
        .agg(["count", "size"])
        .reset_index()
    )
    groupby_df[("Stim eye - Size Mm", "count/size ratio")] = (
        groupby_df[("Stim eye - Size Mm", "count")]
        / groupby_df[("Stim eye - Size Mm", "size")]
    )
    groupby_df[("Stim eye - Size Mm", "more than 50%")] = (
        groupby_df[("Stim eye - Size Mm", "count/size ratio")] >= 0.5
    )
    trial_stats_dict[str(participant_id)] = (
        groupby_df[
            [
                ("Block", ""),
                ("Trial no", ""),
                ("Trial type", ""),
                ("Stim eye - Size Mm", "count/size ratio"),
                ("Stim eye - Size Mm", "more than 50%"),
            ]
        ]
        .groupby([("Block", ""), ("Trial type", "")])
        .agg(
            {
                ("Stim eye - Size Mm", "count/size ratio"): [
                    "mean",
                    "std",
                    "min",
                    "max",
                ],
                ("Stim eye - Size Mm", "more than 50%"): "sum",
                ("Trial no", ""): "count",
            }
        )
        .reset_index()
    )
    
    
    
    data_df_new = []
    groupby_df = []

### Participant 200

In [134]:
print('Statistics for whole trials')
trial_stats_dict["200"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.994083  0.002798  0.989796  0.997963   
1      0        l-m           0.996124  0.002625  0.991837  1.000000   
2      0        lms           0.996565  0.002358  0.991837  1.000000   
3      0        mel           0.995511  0.003161  0.989796  1.000000   
4      0          s           0.996124  0.003656  0.989796  1.000000   
5      1       flux           0.995918  0.004906  0.983673  1.000000   
6      1        l-m           0.996736  0.002755  0.991837  1.000000   
7      1        lms           0.995544  0.002325  0.989796  0.997963   
8      1        mel           0.997754  0.002247  0.993878  1.000000   
9      1          s           0.995102  0.001974  0.991820  0.997959   
10     2       flux           0.990212  0.008742  0.975510  0.997959   
11     2        l-m           0.995306  0.005856  0.981633  1.000000   
12     2        lms           0.996563  0.002532  0.991837  1.000000   
13     2        mel           0.991839  0.007635  0.973469  1.000000   
14     2          s           0.992041  0.009785  0.969388  1.000000   
15     3       flux           0.997756  0.001787  0.993878  1.000000   
16     3        l-m           0.997960  0.001666  0.995918  1.000000   
17     3        lms           0.997158  0.002186  0.993865  1.000000   
18     3        mel           0.996939  0.002205  0.993878  1.000000   
19     3          s           0.995714  0.003528  0.989796  1.000000   
20     4       flux           0.995918  0.002151  0.993878  1.000000   
21     4        l-m           0.993060  0.004104  0.987730  0.997959   
22     4        lms           0.996751  0.003356  0.989796  1.000000   
23     4        mel           0.996940  0.001983  0.993878  1.000000   
24     4          s           0.995102  0.006679  0.977551  1.000000   
25     5       flux           0.991229  0.007993  0.975510  0.997963   
26     5        l-m           0.989798  0.011977  0.957143  0.997959   
27     5        lms           0.993286  0.006016  0.979633  0.998043   
28     5        mel           0.991432  0.005339  0.981633  0.997959   
29     5          s           0.987953  0.007963  0.971429  0.997959   
30     6       flux           0.994286  0.004388  0.987755  1.000000   
31     6        l-m           0.988776  0.007590  0.973469  0.997959   
32     6        lms           0.990699  0.009896  0.965377  0.998043   
33     6        mel           0.994286  0.005593  0.981633  1.000000   
34     6          s           0.973061  0.062131  0.797959  1.000000   
35     7       flux           0.998163  0.002626  0.991837  1.000000   
36     7        l-m           0.996531  0.002367  0.993878  1.000000   
37     7        lms           0.998204  0.001950  0.994129  1.000000   
38     7        mel           0.996735  0.002395  0.993878  1.000000   
39     7          s           0.994696  0.003219  0.989796  0.997959   
40     8       flux           0.994286  0.006821  0.975510  1.000000   
41     8        l-m           0.997007  0.001868  0.993878  1.000000   
42     8        lms           0.933630  0.239184  0.069131  1.000000   
43     8        mel           0.994968  0.004353  0.983673  1.000000   
44     8          s           0.993881  0.004560  0.983673  1.000000   
45     9       flux           0.981047  0.022872  0.940816  1.000000   
46     9        l-m           0.981243  0.018469  0.944672  0.997963   
47     9        lms           0.981908  0.022284  0.935091  0.998043   
48     9        mel           0.986914  0.011726  0.963039  0.997963   
49     9          s           0.984858  0.017904  0.956967  1.000000   
50    10       flux           0.996529  0.003203  0.989775  1.000000   
51    10        l-m           0.997552  0.001875  0.993878  1.000000   
52    10        lms           0.994054  0.0

### Participant 201

In [135]:
print('Statistics for whole trials')
trial_stats_dict["201"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.899625  0.098756  0.757143  0.996086   
1      0        l-m           0.892653  0.158026  0.612245  0.995918   
2      0        lms           0.746122  0.262880  0.402041  0.983673   
3      0        mel           0.720408  0.181925  0.504082  0.948980   
4      0          s           0.804889  0.171056  0.604082  0.987755   
5      1       flux           0.966166  0.025574  0.910156  0.991837   
6      1        l-m           0.971236  0.016705  0.934827  0.989796   
7      1        lms           0.964098  0.028286  0.916327  0.997959   
8      1        mel           0.964975  0.026377  0.896907  0.989796   
9      1          s           0.944200  0.048572  0.836364  0.993878   
10     2       flux           0.981474  0.021498  0.924490  1.000000   
11     2        l-m           0.986122  0.012131  0.959184  0.997963   
12     2        lms           0.979038  0.023064  0.939335  1.000000   
13     2        mel           0.976529  0.017643  0.934694  0.993878   
14     2          s           0.983272  0.012332  0.961303  1.000000   
15     3       flux           0.970846  0.018271  0.942857  0.997959   
16     3        l-m           0.966735  0.031807  0.924490  0.997959   
17     3        lms           0.968257  0.021230  0.938650  0.997959   
18     3        mel           0.977743  0.024298  0.934560  1.000000   
19     3          s           0.977357  0.025240  0.910204  0.995910   
20     4       flux           0.904115  0.115386  0.636735  1.000000   
21     4        l-m           0.957578  0.049349  0.863544  1.000000   
22     4        lms           0.884648  0.174232  0.464844  0.997959   
23     4        mel           0.934915  0.099408  0.665306  0.997959   
24     4          s           0.940862  0.071695  0.784114  1.000000   
25     5       flux           0.793595  0.202697  0.387755  0.984344   
26     5        l-m           0.675156  0.304703  0.246939  0.965306   
27     5        lms           0.734665  0.213463  0.399217  0.985714   
28     5        mel           0.776091  0.188370  0.453061  1.000000   
29     5          s           0.772245  0.251399  0.242857  0.989796   
30     6       flux           0.981887  0.010764  0.963265  0.991837   
31     6        l-m           0.964286  0.029765  0.910204  0.993878   
32     6        lms           0.972923  0.025755  0.918367  1.000000   
33     6        mel           0.976939  0.017948  0.936735  0.991837   
34     6          s           0.956537  0.024152  0.914286  0.981633   
35     7       flux           0.965747  0.037453  0.897959  0.997963   
36     7        l-m           0.957551  0.070734  0.769388  1.000000   
37     7        lms           0.927472  0.103897  0.702041  0.995918   
38     7        mel           0.932052  0.098894  0.718367  0.993878   
39     7          s           0.900621  0.156138  0.553061  0.995918   
40     8       flux           0.994094  0.003542  0.989775  0.997963   
41     8        l-m           0.989185  0.018779  0.936735  0.997959   
42     8        lms           0.991646  0.007789  0.975510  1.000000   
43     8        mel           0.991836  0.011900  0.959184  1.000000   
44     8          s           0.984904  0.019716  0.948980  1.000000   
45     9       flux           0.986966  0.019256  0.946939  1.000000   
46     9        l-m           0.996736  0.004215  0.985714  1.000000   
47     9        lms           0.993078  0.003237  0.987755  0.997959   
48     9        mel           0.990000  0.012923  0.965306  1.000000   
49     9          s           0.992857  0.009963  0.965306  1.000000   
50    10       flux           0.981693  0.016148  0.942974  0.997955   
51    10        l-m           0.983070  0.013310  0.955193  0.995910   
52    10        lms           0.974641  0.0

### Participant 202

In [136]:
print('Statistics for whole trials')
trial_stats_dict["202"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.975149  0.036962  0.891837  1.000000   
1      0        l-m           0.991433  0.007377  0.977551  1.000000   
2      0        lms           0.984924  0.022598  0.926531  0.997959   
3      0        mel           0.981430  0.030582  0.904082  1.000000   
4      0          s           0.984491  0.011397  0.965235  0.997959   
5      1       flux           0.967551  0.051111  0.848980  1.000000   
6      1        l-m           0.952252  0.070436  0.785714  1.000000   
7      1        lms           0.971091  0.068459  0.780041  1.000000   
8      1        mel           0.959182  0.122675  0.610204  1.000000   
9      1          s           0.963265  0.083842  0.726531  1.000000   
10     2       flux           0.938293  0.103636  0.659184  1.000000   
11     2        l-m           0.913513  0.146338  0.606122  1.000000   
12     2        lms           0.876782  0.167051  0.492872  1.000000   
13     2        mel           0.961015  0.063527  0.787755  1.000000   
14     2          s           0.923465  0.151384  0.534694  1.000000   
15     3       flux           0.981458  0.017661  0.942974  0.997959   
16     3        l-m           0.992448  0.012285  0.959184  1.000000   
17     3        lms           0.982449  0.017802  0.953061  0.995927   
18     3        mel           0.991224  0.009954  0.973469  1.000000   
19     3          s           0.992654  0.005945  0.979592  0.997959   
20     4       flux           0.978624  0.020755  0.940816  1.000000   
21     4        l-m           0.990411  0.005448  0.981633  0.997959   
22     4        lms           0.973086  0.035900  0.879592  0.993890   
23     4        mel           0.976529  0.036649  0.887755  1.000000   
24     4          s           0.970635  0.036125  0.902240  0.997959   
25     5       flux           0.924434  0.101106  0.751020  1.000000   
26     5        l-m           0.922515  0.110939  0.678208  0.997963   
27     5        lms           0.946568  0.064454  0.844898  1.000000   
28     5        mel           0.953490  0.054207  0.834694  0.997959   
29     5          s           0.933697  0.070398  0.783673  0.997959   
30     6       flux           0.948003  0.090248  0.720408  0.995918   
31     6        l-m           0.920194  0.145281  0.577551  1.000000   
32     6        lms           0.963349  0.057558  0.824847  0.995918   
33     6        mel           0.986736  0.012953  0.965306  1.000000   
34     6          s           0.953265  0.055303  0.857143  0.997959   
35     7       flux           0.991277  0.014872  0.951120  1.000000   
36     7        l-m           0.965102  0.057574  0.848980  1.000000   
37     7        lms           0.994141  0.004744  0.985714  1.000000   
38     7        mel           0.995103  0.004634  0.987755  1.000000   
39     7          s           0.994694  0.005946  0.983673  1.000000   
40     8       flux           0.981904  0.019356  0.940816  1.000000   
41     8        l-m           0.990003  0.016783  0.944898  0.997959   
42     8        lms           0.979498  0.028132  0.916327  1.000000   
43     8        mel           0.983676  0.027617  0.914286  1.000000   
44     8          s           0.985092  0.022973  0.926531  1.000000   
45     9       flux           0.997559  0.003013  0.989796  1.000000   
46     9        l-m           0.997755  0.002443  0.993878  1.000000   
47     9        lms           0.997790  0.002539  0.992188  1.000000   
48     9        mel           0.996938  0.002406  0.993878  1.000000   
49     9          s           0.993880  0.012283  0.959184  1.000000   
50    10       flux           0.767655  0.318716  0.073469  1.000000   
51    10        l-m           0.697346  0.336796  0.073469  1.000000   
52    10        lms           0.766620  0.3

### Participant 204

In [137]:
print('Statistics for whole trials')
trial_stats_dict["204"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.569717  0.209337  0.271429  0.870637   
1      0        l-m           0.590113  0.202946  0.204082  0.877551   
2      0        lms           0.666084  0.146006  0.434694  0.914460   
3      0        mel           0.594613  0.165200  0.365306  0.848980   
4      0          s           0.588514  0.260018  0.279022  0.951020   
5      1       flux           0.854819  0.112773  0.622309  0.977551   
6      1        l-m           0.810083  0.159986  0.432653  0.963190   
7      1        lms           0.762762  0.233103  0.285714  0.981633   
8      1        mel           0.746169  0.252697  0.354379  0.989775   
9      1          s           0.810137  0.193114  0.423625  0.985743   
10     2       flux           0.826515  0.202663  0.359184  0.981633   
11     2        l-m           0.918779  0.049934  0.848980  0.989796   
12     2        lms           0.849181  0.207171  0.452138  0.991837   
13     2        mel           0.768776  0.186876  0.497959  0.979592   
14     2          s           0.883041  0.115306  0.589796  0.985714   
15     3       flux           0.807956  0.172179  0.491837  0.969388   
16     3        l-m           0.828701  0.073175  0.678208  0.940816   
17     3        lms           0.800935  0.100451  0.597959  0.922449   
18     3        mel           0.861817  0.074178  0.679592  0.967347   
19     3          s           0.828827  0.079265  0.677551  0.928571   
20     4       flux           0.758216  0.146284  0.551020  0.902041   
21     4        l-m           0.694286  0.193190  0.410204  0.932653   
22     4        lms           0.740390  0.249760  0.124490  0.924490   
23     4        mel           0.735101  0.219163  0.248980  0.959184   
24     4          s           0.802289  0.088114  0.636735  0.938900   
25     5       flux           0.691275  0.134315  0.502041  0.948980   
26     5        l-m           0.702740  0.189079  0.285714  0.893878   
27     5        lms           0.604277  0.212043  0.222904  0.869388   
28     5        mel           0.723686  0.068978  0.638776  0.828571   
29     5          s           0.724683  0.181658  0.269388  0.875510   
30     6       flux           0.655646  0.228207  0.316406  0.930612   
31     6        l-m           0.705950  0.173075  0.473469  0.973469   
32     6        lms           0.569565  0.234205  0.120163  0.802041   
33     6        mel           0.744082  0.213168  0.287755  0.967347   
34     6          s           0.618400  0.215095  0.164969  0.873469   
35     7       flux           0.515712  0.188976  0.234987  0.921875   
36     7        l-m           0.469499  0.220034  0.038462  0.830612   
37     7        lms           0.529585  0.273486  0.038462  0.940695   
38     7        mel           0.521180  0.251298  0.111406  0.900000   
39     7          s           0.508475  0.242542  0.161458  0.879592   
40     8       flux           0.611091  0.222072  0.299389  0.947059   
41     8        l-m           0.704193  0.146911  0.471429  0.916497   
42     8        lms           0.703646  0.162480  0.413442  0.952965   
43     8        mel           0.679339  0.121345  0.469388  0.840816   
44     8          s           0.779660  0.155787  0.487755  0.961303   
45     9       flux           0.679711  0.241659  0.118367  0.951076   
46     9        l-m           0.650118  0.215138  0.263265  0.891837   
47     9        lms           0.628091  0.253238  0.067347  0.857143   
48     9        mel           0.683018  0.155371  0.364562  0.873469   
49     9          s           0.618524  0.271443  0.063395  0.934694   
50    10       flux           0.913754  0.043072  0.809816  0.965306   
51    10        l-m           0.903060  0.026056  0.861224  0.942857   
52    10        lms           0.906849  0.0

### Participant 205

In [138]:
print('Statistics for whole trials')
trial_stats_dict["205"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.495864  0.158681  0.253061  0.669388   
1      0        l-m           0.404861  0.250675  0.079592  0.806122   
2      0        lms           0.395130  0.235073  0.022449  0.743381   
3      0        mel           0.235059  0.194711  0.000000  0.532653   
4      0          s           0.306783  0.204738  0.085714  0.763265   
5      1       flux           0.555844  0.251875  0.281059  0.897959   
6      1        l-m           0.519352  0.225571  0.162933  0.853061   
7      1        lms           0.503176  0.360995  0.000000  0.908163   
8      1        mel           0.461108  0.246092  0.000000  0.748980   
9      1          s           0.534323  0.244534  0.169388  0.934694   
10     2       flux           0.485553  0.353413  0.126273  0.938776   
11     2        l-m           0.462329  0.272138  0.000000  0.826531   
12     2        lms           0.425588  0.315317  0.040816  0.920408   
13     2        mel           0.358980  0.242937  0.100000  0.867347   
14     2          s           0.417143  0.374303  0.018367  0.946939   
15     3       flux           0.486554  0.176181  0.265306  0.853229   
16     3        l-m           0.430418  0.229098  0.171429  0.906122   
17     3        lms           0.571758  0.182428  0.361224  0.889796   
18     3        mel           0.472397  0.271793  0.108163  0.918367   
19     3          s           0.549505  0.209915  0.193878  0.893878   
20     4       flux           0.230746  0.125264  0.063265  0.457143   
21     4        l-m           0.287276  0.269506  0.000000  0.732653   
22     4        lms           0.190485  0.176827  0.000000  0.595918   
23     4        mel           0.272393  0.148576  0.063265  0.567347   
24     4          s           0.232041  0.166984  0.000000  0.469388   
25     5       flux           0.328802  0.256278  0.000000  0.687755   
26     5        l-m           0.391137  0.247151  0.000000  0.742857   
27     5        lms           0.423562  0.228200  0.100000  0.691837   
28     5        mel           0.456073  0.245174  0.141104  0.859184   
29     5          s           0.481786  0.209349  0.183673  0.836735   
30     6       flux           0.590616  0.242655  0.197959  0.872798   
31     6        l-m           0.624114  0.254790  0.242857  0.918367   
32     6        lms           0.628696  0.229980  0.226531  0.938776   
33     6        mel           0.613173  0.225387  0.165306  0.859184   
34     6          s           0.601517  0.210048  0.240326  0.944898   
35     7       flux           0.740408  0.127076  0.593878  0.871429   
36     7        l-m           0.777551  0.058422  0.720408  0.867347   
37     7        lms           0.772963  0.151045  0.607362  0.962818   
38     7        mel           0.565714  0.207730  0.293878  0.808163   
39     7          s           0.662564  0.110415  0.508163  0.806122   
40     8       flux           0.799685  0.089960  0.627291  0.932653   
41     8        l-m           0.690472  0.133004  0.469388  0.865306   
42     8        lms           0.752574  0.159534  0.444898  0.933464   
43     8        mel           0.600612  0.191492  0.340816  0.924490   
44     8          s           0.666378  0.183653  0.395918  0.928717   
45     9       flux           0.508504  0.248348  0.000000  0.820041   
46     9        l-m           0.527650  0.185198  0.255102  0.788187   
47     9        lms           0.605597  0.236253  0.140816  0.869388   
48     9        mel           0.661513  0.149671  0.442857  0.940937   
49     9          s           0.503918  0.207903  0.185336  0.744898   
50    10       flux           0.541417  0.317281  0.073469  0.930470   
51    10        l-m           0.442855  0.273650  0.073469  0.934694   
52    10        lms           0.561578  0.3

### Participant 206

In [139]:
print('Statistics for whole trials')
trial_stats_dict["206"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.924330  0.082927  0.700611  0.987755   
1      0        l-m           0.935930  0.054631  0.834694  0.993890   
2      0        lms           0.948182  0.026104  0.904082  0.981633   
3      0        mel           0.925715  0.039129  0.844898  0.969388   
4      0          s           0.923828  0.089805  0.716327  0.983673   
5      1       flux           0.649860  0.131920  0.569191  0.997959   
6      1        l-m           0.615238  0.136350  0.532213  0.995918   
7      1        lms           0.666730  0.173471  0.540260  0.997959   
8      1        mel           0.675637  0.220357  0.426735  0.989796   
9      1          s           0.666495  0.168562  0.566580  0.987755   
10     2       flux           0.965135  0.020191  0.930612  0.994141   
11     2        l-m           0.966327  0.016163  0.944898  0.995918   
12     2        lms           0.964577  0.044604  0.842857  0.997963   
13     2        mel           0.928972  0.145388  0.516327  0.987755   
14     2          s           0.928980  0.101147  0.677551  0.997963   
15     3       flux           0.909183  0.071279  0.810176  0.987780   
16     3        l-m           0.866327  0.149122  0.518367  0.991837   
17     3        lms           0.840542  0.206087  0.344196  0.987755   
18     3        mel           0.879615  0.101767  0.671429  0.979633   
19     3          s           0.843151  0.174851  0.437628  0.989796   
20     4       flux           0.929042  0.091158  0.689796  0.986301   
21     4        l-m           0.959602  0.031330  0.902041  0.991837   
22     4        lms           0.958583  0.024718  0.922449  0.993878   
23     4        mel           0.958413  0.039460  0.890020  0.997959   
24     4          s           0.964694  0.032140  0.900000  0.995918   
25     5       flux           0.799030  0.170939  0.578732  0.985714   
26     5        l-m           0.821032  0.129392  0.579592  0.961224   
27     5        lms           0.873519  0.082493  0.744898  0.953061   
28     5        mel           0.885102  0.074346  0.726531  0.987755   
29     5          s           0.871435  0.094450  0.710204  0.969450   
30     6       flux           0.851550  0.155881  0.470468  0.989817   
31     6        l-m           0.849388  0.097018  0.626531  0.934827   
32     6        lms           0.843084  0.121638  0.550102  0.942857   
33     6        mel           0.769257  0.132608  0.598778  0.961224   
34     6          s           0.844101  0.097977  0.677551  0.957143   
35     7       flux           0.789893  0.246981  0.313112  0.995918   
36     7        l-m           0.869191  0.206519  0.320408  0.995918   
37     7        lms           0.752871  0.319152  0.189796  0.993878   
38     7        mel           0.689694  0.313087  0.181633  0.981633   
39     7          s           0.765511  0.274821  0.365306  0.995918   
40     8       flux           0.969796  0.018737  0.934827  0.993890   
41     8        l-m           0.957556  0.026049  0.906122  0.997959   
42     8        lms           0.955189  0.032976  0.891837  0.991837   
43     8        mel           0.979601  0.012125  0.948980  0.991853   
44     8          s           0.957546  0.061765  0.789796  0.997959   
45     9       flux           0.978794  0.033089  0.902041  1.000000   
46     9        l-m           0.991423  0.009718  0.973415  1.000000   
47     9        lms           0.994312  0.005508  0.981633  1.000000   
48     9        mel           0.991224  0.013810  0.965306  1.000000   
49     9          s           0.985714  0.029590  0.902041  1.000000   
50    10       flux           0.944091  0.065153  0.812245  0.995918   
51    10        l-m           0.918803  0.094676  0.751020  1.000000   
52    10        lms           0.965752  0.0

### Participant 207

In [140]:
print('Statistics for whole trials')
trial_stats_dict["207"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.822488  0.188029  0.391837  0.965306   
1      0        l-m           0.911843  0.073756  0.775510  0.975510   
2      0        lms           0.839747  0.207554  0.301426  0.975510   
3      0        mel           0.834939  0.258387  0.143149  0.977551   
4      0          s           0.894490  0.130924  0.548980  0.973469   
5      1       flux           0.913009  0.022104  0.869388  0.940937   
6      1        l-m           0.928163  0.028406  0.891837  0.971429   
7      1        lms           0.912122  0.037359  0.840816  0.963340   
8      1        mel           0.894529  0.097750  0.633401  0.955193   
9      1          s           0.924908  0.023078  0.879592  0.953157   
10     2       flux           0.556640  0.264792  0.000000  0.897959   
11     2        l-m           0.762473  0.131454  0.551020  0.908163   
12     2        lms           0.702516  0.230548  0.144898  0.885714   
13     2        mel           0.717959  0.262613  0.000000  0.948980   
14     2          s           0.659384  0.209532  0.228571  0.885714   
15     3       flux           0.652091  0.215891  0.289628  0.918367   
16     3        l-m           0.612099  0.151942  0.362525  0.843177   
17     3        lms           0.685456  0.127328  0.503067  0.908350   
18     3        mel           0.647347  0.194472  0.338776  0.918367   
19     3          s           0.708717  0.149289  0.432653  0.873469   
20     4       flux           0.220483  0.171768  0.000000  0.593878   
21     4        l-m           0.324245  0.161001  0.083673  0.627291   
22     4        lms           0.365951  0.232856  0.014286  0.710204   
23     4        mel           0.301914  0.180188  0.000000  0.619145   
24     4          s           0.241491  0.222425  0.020367  0.704082   
25     5       flux           0.504244  0.179044  0.226531  0.732653   
26     5        l-m           0.519388  0.198945  0.175510  0.763265   
27     5        lms           0.431024  0.254962  0.000000  0.700000   
28     5        mel           0.526125  0.205030  0.193878  0.838776   
29     5          s           0.418512  0.233974  0.067210  0.822449   
30     6       flux           0.717085  0.238854  0.148980  0.930612   
31     6        l-m           0.722024  0.203499  0.303462  0.912424   
32     6        lms           0.639144  0.244068  0.283673  0.908163   
33     6        mel           0.649852  0.194577  0.218367  0.859184   
34     6          s           0.755535  0.179667  0.322449  0.946939   
35     7       flux           0.679215  0.238950  0.085890  0.957143   
36     7        l-m           0.762963  0.133706  0.575510  0.957055   
37     7        lms           0.716906  0.177852  0.395303  0.948980   
38     7        mel           0.680307  0.270989  0.004682  0.961224   
39     7          s           0.657183  0.310385  0.118367  0.953061   
40     8       flux           0.694590  0.235902  0.195918  0.918367   
41     8        l-m           0.634154  0.328811  0.146939  0.940816   
42     8        lms           0.718041  0.265336  0.055102  0.932653   
43     8        mel           0.645777  0.250937  0.151020  0.934694   
44     8          s           0.656081  0.300508  0.095918  0.914286   
45     9       flux           0.703423  0.299636  0.069246  0.942857   
46     9        l-m           0.870169  0.022755  0.844581  0.908163   
47     9        lms           0.700003  0.238998  0.318367  0.926531   
48     9        mel           0.802720  0.095619  0.610204  0.914286   
49     9          s           0.643836  0.336625  0.061350  0.944898   
50    10       flux           0.606947  0.166856  0.281633  0.810204   
51    10        l-m           0.665399  0.169335  0.408163  0.875256   
52    10        lms           0.489472  0.2

### Participant 209

In [141]:
print('Statistics for whole trials')
trial_stats_dict["209"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.377752  0.176101  0.016327  0.636735   
1      0        l-m           0.343826  0.252102  0.000000  0.787755   
2      0        lms           0.309782  0.245155  0.000000  0.648980   
3      0        mel           0.403583  0.251529  0.030612  0.797959   
4      0          s           0.335325  0.263388  0.000000  0.710794   
5      1       flux           0.406809  0.274448  0.000000  0.979592   
6      1        l-m           0.379692  0.278807  0.073469  0.826531   
7      1        lms           0.442209  0.236634  0.000000  0.900204   
8      1        mel           0.492290  0.295018  0.000000  0.997959   
9      1          s           0.358628  0.287975  0.000000  1.000000   
10     2       flux           0.585496  0.247414  0.155102  0.970646   
11     2        l-m           0.648516  0.276163  0.312245  0.981633   
12     2        lms           0.530950  0.293196  0.075510  0.946939   
13     2        mel           0.592327  0.330929  0.071429  0.959100   
14     2          s           0.542339  0.278571  0.046939  0.908163   
15     3       flux           0.349878  0.301664  0.022495  0.770961   
16     3        l-m           0.221561  0.208408  0.000000  0.683673   
17     3        lms           0.327940  0.305533  0.000000  0.753061   
18     3        mel           0.382546  0.343427  0.000000  0.928425   
19     3          s           0.299739  0.228809  0.053061  0.783673   
20     4       flux           0.459705  0.262402  0.000000  0.885714   
21     4        l-m           0.408013  0.203014  0.059184  0.714868   
22     4        lms           0.485467  0.335364  0.026531  0.983673   
23     4        mel           0.458567  0.296604  0.000000  0.908163   
24     4          s           0.495637  0.307607  0.000000  0.897959   
25     5       flux           0.861780  0.091962  0.667347  0.939335   
26     5        l-m           0.891177  0.050930  0.770961  0.946939   
27     5        lms           0.883370  0.066186  0.755102  0.969388   
28     5        mel           0.874927  0.064729  0.724490  0.934694   
29     5          s           0.829616  0.089047  0.655102  0.912245   
30     6       flux           0.120398  0.165374  0.000000  0.471429   
31     6        l-m           0.063896  0.101386  0.000000  0.332653   
32     6        lms           0.075569  0.108022  0.000000  0.290389   
33     6        mel           0.180878  0.267949  0.000000  0.696538   
34     6          s           0.107873  0.165032  0.000000  0.421589   
35     7       flux           0.057568  0.144278  0.000000  0.461224   
36     7        l-m           0.016732  0.032006  0.000000  0.083673   
37     7        lms           0.043469  0.129087  0.000000  0.410204   
38     7        mel           0.012231  0.020550  0.000000  0.055102   
39     7          s           0.081365  0.109360  0.000000  0.314286   
40     8       flux           0.612741  0.301023  0.285714  0.993878   
41     8        l-m           0.734567  0.212273  0.369388  0.969388   
42     8        lms           0.701079  0.230901  0.334012  0.973469   
43     8        mel           0.710374  0.158433  0.430612  0.987755   
44     8          s           0.675551  0.308463  0.136735  0.967347   
45     9       flux           0.554964  0.209886  0.200000  0.816327   
46     9        l-m           0.529017  0.219484  0.140816  0.817996   
47     9        lms           0.678115  0.171107  0.442270  0.926531   
48     9        mel           0.630527  0.278919  0.016327  0.951020   
49     9          s           0.708488  0.112399  0.541752  0.916327   
50    10       flux           0.661610  0.244064  0.275510  0.918367   
51    10        l-m           0.651007  0.196831  0.326531  0.830612   
52    10        lms           0.583928  0.2

### Participant 210

In [142]:
print('Statistics for whole trials')
trial_stats_dict["210"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.840844  0.076861  0.693878  0.942857   
1      0        l-m           0.807091  0.126136  0.586912  0.940816   
2      0        lms           0.838314  0.116526  0.630612  0.986328   
3      0        mel           0.812419  0.109177  0.622449  0.951020   
4      0          s           0.850204  0.075906  0.726531  0.977551   
5      1       flux           0.554785  0.234602  0.138776  0.823875   
6      1        l-m           0.519868  0.262193  0.034623  0.969450   
7      1        lms           0.586072  0.189775  0.275510  0.932653   
8      1        mel           0.530265  0.176173  0.271429  0.781633   
9      1          s           0.660925  0.125427  0.468432  0.885714   
10     2       flux           0.781267  0.132671  0.485714  0.934694   
11     2        l-m           0.791469  0.153273  0.497959  0.940816   
12     2        lms           0.813555  0.100906  0.606654  0.926680   
13     2        mel           0.748571  0.152953  0.508163  0.904082   
14     2          s           0.794985  0.102194  0.623218  0.932653   
15     3       flux           0.589060  0.297558  0.177551  0.934694   
16     3        l-m           0.554283  0.252697  0.218367  0.936864   
17     3        lms           0.510990  0.341227  0.053061  0.963265   
18     3        mel           0.556675  0.331261  0.081466  0.924490   
19     3          s           0.541984  0.262216  0.151020  0.867347   
20     4       flux           0.490354  0.316838  0.089613  0.955102   
21     4        l-m           0.621803  0.194864  0.248980  0.818737   
22     4        lms           0.444070  0.191285  0.216327  0.814286   
23     4        mel           0.596327  0.205000  0.242857  0.816327   
24     4          s           0.439987  0.194802  0.215886  0.820408   
25     5       flux           0.506389  0.250940  0.236735  0.936735   
26     5        l-m           0.697092  0.243124  0.327394  0.987755   
27     5        lms           0.428743  0.288406  0.048980  0.889796   
28     5        mel           0.493252  0.243705  0.038776  0.832653   
29     5          s           0.589546  0.268599  0.054920  0.961224   
30     6       flux           0.531375  0.263371  0.175510  0.887755   
31     6        l-m           0.674161  0.201857  0.269388  0.910204   
32     6        lms           0.671211  0.267362  0.183673  0.895918   
33     6        mel           0.525248  0.219980  0.236735  0.844898   
34     6          s           0.596430  0.237053  0.224033  0.871690   
35     7       flux           0.388304  0.156079  0.191837  0.650391   
36     7        l-m           0.472397  0.140691  0.230612  0.694501   
37     7        lms           0.556595  0.217128  0.177551  0.851020   
38     7        mel           0.483422  0.261352  0.155102  0.918367   
39     7          s           0.436327  0.210412  0.171429  0.818367   
40     8       flux           0.498267  0.169509  0.300000  0.802041   
41     8        l-m           0.488776  0.194113  0.218367  0.861224   
42     8        lms           0.458197  0.240619  0.000000  0.767347   
43     8        mel           0.389992  0.197752  0.073469  0.649695   
44     8          s           0.419109  0.236757  0.220408  0.979592   
45     9       flux           0.504402  0.256031  0.161224  0.834694   
46     9        l-m           0.746390  0.115352  0.532653  0.955102   
47     9        lms           0.486288  0.209083  0.242857  0.757143   
48     9        mel           0.541039  0.223294  0.277551  0.910387   
49     9          s           0.623038  0.139701  0.377551  0.887526   
50    10       flux           0.408032  0.294655  0.028513  0.892057   
51    10        l-m           0.572049  0.210021  0.268839  0.873469   
52    10        lms           0.306599  0.2

### Participant 211

In [143]:
print('Statistics for whole trials')
trial_stats_dict["211"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.919558  0.048850  0.846939  1.000000   
1      0        l-m           0.955934  0.024663  0.916327  0.989796   
2      0        lms           0.945645  0.047174  0.833659  0.989796   
3      0        mel           0.909796  0.081909  0.718367  0.997959   
4      0          s           0.881837  0.058925  0.783673  0.944898   
5      1       flux           0.696283  0.302559  0.306122  0.992188   
6      1        l-m           0.820416  0.187196  0.424490  0.975510   
7      1        lms           0.781625  0.205079  0.439919  0.985714   
8      1        mel           0.769323  0.264848  0.350305  0.995918   
9      1          s           0.843061  0.156898  0.524490  0.991837   
10     2       flux           0.935225  0.057457  0.834694  0.985714   
11     2        l-m           0.960810  0.020604  0.920408  0.979592   
12     2        lms           0.928695  0.043908  0.844898  0.987780   
13     2        mel           0.957132  0.031806  0.893878  1.000000   
14     2          s           0.927555  0.049410  0.832653  0.979633   
15     3       flux           0.952975  0.044495  0.837838  0.989796   
16     3        l-m           0.952227  0.029760  0.895918  0.983673   
17     3        lms           0.928443  0.049994  0.837067  0.987755   
18     3        mel           0.944590  0.032080  0.898760  0.981633   
19     3          s           0.947388  0.052399  0.837067  0.993878   
20     4       flux           0.887343  0.096429  0.648980  0.969325   
21     4        l-m           0.808349  0.300924  0.124236  0.979592   
22     4        lms           0.927445  0.055568  0.806122  0.985743   
23     4        mel           0.789804  0.273712  0.146939  0.987755   
24     4          s           0.887751  0.150803  0.483673  0.995910   
25     5       flux           0.860670  0.094044  0.723014  0.975510   
26     5        l-m           0.822727  0.113838  0.603272  0.963265   
27     5        lms           0.895969  0.081583  0.789796  0.983640   
28     5        mel           0.848165  0.113804  0.624490  0.959184   
29     5          s           0.835185  0.128259  0.613035  0.969388   
30     6       flux           0.795207  0.106482  0.587755  0.914286   
31     6        l-m           0.721633  0.185414  0.522449  0.957143   
32     6        lms           0.817241  0.139284  0.510204  0.975510   
33     6        mel           0.762045  0.144008  0.604888  0.993878   
34     6          s           0.852662  0.129402  0.616327  0.981633   
35     7       flux           0.838081  0.159312  0.477551  0.989796   
36     7        l-m           0.783577  0.224215  0.410204  0.993890   
37     7        lms           0.857433  0.087449  0.740286  0.987730   
38     7        mel           0.902680  0.054615  0.832653  0.991837   
39     7          s           0.806939  0.168390  0.500000  0.995918   
40     8       flux           0.938681  0.047312  0.843177  0.975560   
41     8        l-m           0.943646  0.083157  0.728571  1.000000   
42     8        lms           0.954066  0.042299  0.861507  0.995918   
43     8        mel           0.932389  0.067777  0.790224  0.993890   
44     8          s           0.938032  0.049385  0.828921  0.991837   
45     9       flux           0.987382  0.023332  0.924490  1.000000   
46     9        l-m           0.979805  0.026407  0.920570  1.000000   
47     9        lms           0.983836  0.015226  0.963265  1.000000   
48     9        mel           0.975511  0.035843  0.895918  1.000000   
49     9          s           0.986327  0.012397  0.963265  1.000000   
50    10       flux           0.964601  0.032078  0.885714  0.997955   
51    10        l-m           0.979800  0.010817  0.969450  0.995918   
52    10        lms           0.971107  0.0

### Participant 212

In [144]:
print('Statistics for whole trials')
trial_stats_dict["212"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.791155  0.289139  0.238776  1.000000   
1      0        l-m           0.844504  0.230958  0.383673  0.997959   
2      0        lms           0.735945  0.247511  0.262729  0.993878   
3      0        mel           0.718458  0.282049  0.167347  0.997959   
4      0          s           0.775149  0.273113  0.242857  0.995918   
5      1       flux           0.682777  0.276188  0.165306  1.000000   
6      1        l-m           0.639464  0.325117  0.061224  0.997959   
7      1        lms           0.675079  0.342964  0.077551  1.000000   
8      1        mel           0.634091  0.352386  0.057026  0.997959   
9      1          s           0.630002  0.416496  0.000000  0.997959   
10     2       flux           0.847234  0.313745  0.000000  1.000000   
11     2        l-m           0.948792  0.113511  0.632653  1.000000   
12     2        lms           0.874146  0.297586  0.034836  0.997959   
13     2        mel           0.876925  0.309370  0.000000  1.000000   
14     2          s           0.718675  0.409610  0.000000  1.000000   
15     3       flux           0.994710  0.002590  0.991837  0.997959   
16     3        l-m           0.974899  0.040224  0.881633  1.000000   
17     3        lms           0.976816  0.039155  0.896130  1.000000   
18     3        mel           0.995921  0.004604  0.983707  1.000000   
19     3          s           0.993270  0.003972  0.985714  0.997963   
20     4       flux           0.952653  0.075284  0.797959  1.000000   
21     4        l-m           0.988982  0.010979  0.961224  0.997959   
22     4        lms           0.993485  0.005518  0.981633  1.000000   
23     4        mel           0.992858  0.005950  0.981633  0.997963   
24     4          s           0.983470  0.035328  0.883673  1.000000   
25     5       flux           0.983307  0.019204  0.938776  1.000000   
26     5        l-m           0.983065  0.013843  0.959184  1.000000   
27     5        lms           0.989410  0.009443  0.969325  1.000000   
28     5        mel           0.991227  0.008871  0.975510  1.000000   
29     5          s           0.968165  0.033094  0.897959  0.997959   
30     6       flux           0.739475  0.221709  0.242661  0.997959   
31     6        l-m           0.566931  0.311502  0.171429  0.936735   
32     6        lms           0.786718  0.176949  0.495918  0.997959   
33     6        mel           0.758604  0.223598  0.340816  0.995918   
34     6          s           0.834286  0.152286  0.534694  0.995918   
35     7       flux           0.938174  0.174265  0.442857  1.000000   
36     7        l-m           0.899807  0.145764  0.616327  1.000000   
37     7        lms           0.992683  0.005542  0.981670  0.997959   
38     7        mel           0.932403  0.113915  0.672802  1.000000   
39     7          s           0.982647  0.032343  0.893878  1.000000   
40     8       flux           0.987577  0.019689  0.940816  1.000000   
41     8        l-m           0.940815  0.095862  0.700000  0.997959   
42     8        lms           0.994330  0.005556  0.985714  1.000000   
43     8        mel           0.991233  0.012147  0.959267  1.000000   
44     8          s           0.986124  0.033971  0.889796  1.000000   
45     9       flux           0.951316  0.079172  0.738776  0.993890   
46     9        l-m           0.834606  0.230789  0.438776  0.995918   
47     9        lms           0.942880  0.108375  0.661224  1.000000   
48     9        mel           0.952862  0.086128  0.712245  0.995918   
49     9          s           0.970989  0.035493  0.881633  0.995918   
50    10       flux           0.951689  0.070970  0.769388  0.997959   
51    10        l-m           0.691803  0.408236  0.032787  0.997963   
52    10        lms           0.904890  0.2

### Participant 213

In [145]:
print('Statistics for whole trials')
trial_stats_dict["213"]

Statistics for whole trials


Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.671325  0.279352  0.000000  0.997959   
1      0        l-m           0.727378  0.368059  0.000000  0.989796   
2      0        lms           0.501788  0.388463  0.000000  0.928717   
3      0        mel           0.524490  0.455060  0.000000  0.942857   
4      0          s           0.609268  0.351218  0.067347  0.993878   
5      1       flux           0.975512  0.023402  0.936735  1.000000   
6      1        l-m           0.886169  0.113352  0.636735  0.993890   
7      1        lms           0.946653  0.065299  0.786151  0.997959   
8      1        mel           0.960628  0.033987  0.894094  0.997959   
9      1          s           0.927755  0.146039  0.522449  1.000000   
10     2       flux           0.915729  0.149665  0.612245  0.998047   
11     2        l-m           0.934101  0.080155  0.757143  0.997955   
12     2        lms           0.930006  0.172176  0.444898  1.000000   
13     2        mel           0.814633  0.309202  0.189796  1.000000   
14     2          s           0.903878  0.178751  0.446939  1.000000   
15     3       flux           0.917331  0.138348  0.538776  1.000000   
16     3        l-m           0.811783  0.127548  0.600000  0.989796   
17     3        lms           0.872479  0.122603  0.648980  1.000000   
18     3        mel           0.902227  0.167941  0.455102  1.000000   
19     3          s           0.835272  0.219541  0.334012  1.000000   
20     4       flux           0.915777  0.131751  0.573469  0.995918   
21     4        l-m           0.903269  0.154243  0.508163  0.993878   
22     4        lms           0.884837  0.151589  0.558045  1.000000   
23     4        mel           0.922496  0.085446  0.732653  0.991837   
24     4          s           0.947349  0.079931  0.755102  0.997959   
25     5       flux           0.717261  0.234421  0.283673  0.997959   
26     5        l-m           0.707184  0.265489  0.285714  0.987780   
27     5        lms           0.728579  0.210883  0.393075  0.995918   
28     5        mel           0.693878  0.290123  0.063265  0.975510   
29     5          s           0.638019  0.350927  0.069388  0.967413   
30     6       flux           0.760153  0.251366  0.193878  0.977551   
31     6        l-m           0.521696  0.293844  0.077551  1.000000   
32     6        lms           0.679133  0.235209  0.316327  0.924644   
33     6        mel           0.615097  0.293348  0.144898  0.997959   
34     6          s           0.499926  0.341478  0.034694  1.000000   
35     7       flux           0.677483  0.270263  0.126273  0.997959   
36     7        l-m           0.525376  0.292810  0.100000  0.902041   
37     7        lms           0.559316  0.326833  0.051020  0.963265   
38     7        mel           0.426943  0.267808  0.124490  0.900000   
39     7          s           0.712126  0.155258  0.495918  0.918367   
40     8       flux           0.519949  0.404144  0.024490  1.000000   
41     8        l-m           0.410737  0.343120  0.000000  0.977551   
42     8        lms           0.480392  0.390348  0.014286  0.995918   
43     8        mel           0.427755  0.361551  0.022449  1.000000   
44     8          s           0.549751  0.419362  0.028571  1.000000   
45     9       flux           0.643231  0.261145  0.210204  0.977551   
46     9        l-m           0.658408  0.329158  0.188139  0.997959   
47     9        lms           0.840828  0.210357  0.276639  1.000000   
48     9        mel           0.827367  0.240899  0.328571  0.995927   
49     9          s           0.801944  0.214737  0.385714  0.995918   
50    10       flux           0.870630  0.163604  0.497959  0.998043   
51    10        l-m           0.777347  0.126992  0.577551  0.997959   
52    10        lms           0.919720  0.1

In [147]:
blocks = np.arange(11)


def find_consecutive_nans(data_df):
    pass
    